In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
def update_player_weekly_stats():
    
    urls = ['https://www.pro-football-reference.com/players/A/',
       'https://www.pro-football-reference.com/players/B/',
       'https://www.pro-football-reference.com/players/C/',
       'https://www.pro-football-reference.com/players/D/',
       'https://www.pro-football-reference.com/players/E/',
       'https://www.pro-football-reference.com/players/F/',
       'https://www.pro-football-reference.com/players/G/',
       'https://www.pro-football-reference.com/players/H/',
       'https://www.pro-football-reference.com/players/I/',
       'https://www.pro-football-reference.com/players/J/',
       'https://www.pro-football-reference.com/players/K/',
       'https://www.pro-football-reference.com/players/L/',
       'https://www.pro-football-reference.com/players/M/',
       'https://www.pro-football-reference.com/players/N/',
        'https://www.pro-football-reference.com/players/O/',
       'https://www.pro-football-reference.com/players/P/',
       'https://www.pro-football-reference.com/players/Q/',
       'https://www.pro-football-reference.com/players/R/',
       'https://www.pro-football-reference.com/players/S/',
       'https://www.pro-football-reference.com/players/T/',
       'https://www.pro-football-reference.com/players/U/',
       'https://www.pro-football-reference.com/players/V/',
       'https://www.pro-football-reference.com/players/W/',
       'https://www.pro-football-reference.com/players/X/',
       'https://www.pro-football-reference.com/players/Y/',
       'https://www.pro-football-reference.com/players/Z/']
    
    
    l = []
    for i in range(len(urls)):
        time.sleep(5)

        url = urls[i]
        data = requests.get(url).text
        soup = BeautifulSoup(data, 'lxml')
        tags = soup.find_all('p')
        tags = tags[:-12]
       # df_dropped_last_n = df.iloc[:-n]
       # tags = tags.astype(str)
        [l.append(x) for x in tags] 
       # l.append(tags)
    
    tags_df = pd.DataFrame(l, columns = ['Player','Position'])

    test = tags_df
    #test.head()
    test = pd.DataFrame(test)
    #test.columns
    test['colB'] = test['Player'].replace('[', '')
    
    
    test['Player'] = test['Player'].astype(str)
    test['Position'] = test['Position'].astype(str)
    #test.head()
    new = test['Player'].str.split('"', expand=True)
    
    
    new = new.drop(columns = [0])
    new[1] = new[1].str.replace('.htm',"")

    new['Gamelog'] = 'https://www.pro-football-reference.com' + new[1] + '/gamelog/'
    #new2 = new[2].str.split()
    new['Player'] = new[2].str.replace('</a>',"")
    new['Player'] = new['Player'].str.replace('</b>',"")
    new['Player'] = new['Player'].str.replace('>',"")

    new2 = new['Player'].str.split(' ', expand=True)
    
    pos_list = ['(WR)', '(LB)', '(RB)', '(DE)', '(S)', '(OT)', '(DB)', '(OLB)', '(OL)',
           '(QB)', '(P)', '(G)', '(CB)', '(DL)', '(FS)', '(K)', '(OG)',
           '(EDGE)', '(DT)', '(TE)', '(T)', '(NT)', '(FB)', '(SS)', '(LS)',
           '(ILB)', '(OG)', '(C)', '(DT)', '(DL)']

    new2['position'] = np.where(new2[2].isin(pos_list), new2[2], 
                           np.where(new2[3].isin(pos_list), new2[3], ""))
    new2['player'] = np.where(((~new2[3].isin(pos_list)) & (new2[3].notnull())), new2[0] + " " + new2[1] + " " + new2[2] + " " + new2[3],
                         np.where(((~new2[2].isin(pos_list)) & (new2[2].notnull())), new2[0] + " " + new2[1] + " " + new2[2],
                                 new2[0] + " " + new2[1]))
    
    
    new['Player'] = new2['player']
    
    test['Gamelog'] = new['Gamelog']
    test['Player'] = new['Player']
    test['Position_2'] = new2['position']
    
    pos = test['Position'].str.split(' ', expand=True)
    
    pos['position'] = np.where((pos[1].str.contains('19')) | (pos[1].str.contains('20')),"",
                           pos[1])

    pos['playing_period'] = np.where((pos[1].str.contains('19')) | (pos[1].str.contains('20')), pos[1],
                                pos[2])
    
    test['Position_1'] = pos['position']
    test['Playing_Prd'] = pos['playing_period']
    test['Position'] = np.where(test['Position_1'] != "", test['Position_1'],
                           test['Position_2'])
    test['Position'] = test['Position'].str[1:]
    test['Position'] = test['Position'].str[:-1]
    
    play_prd = test['Playing_Prd'].str.split('-', expand=True)
    
    test['Playing_Prd_Strt'] = play_prd[0]
    test['Playing_Prd_End'] = play_prd[1]
    
    players = test[['Gamelog', 'Player','Position',
                'Playing_Prd_Strt','Playing_Prd_End']]

    
    
    players['Playing_Prd_End'] = players['Playing_Prd_End'].astype(int)
    players['Playing_Prd_Strt'] = players['Playing_Prd_Strt'].astype(int)

    players_curr = players[players['Playing_Prd_End'] >= 2012]
    
    
    players_curr = players_curr[players_curr['Position'] != 'TE-C']
    players_curr = players_curr[players_curr['Position'] != 'DE-C']

    players_curr['Position'] = players_curr['Position'].map({'SAF':'S','LT':'T','QB/TE':'TE','G,C':'G',
                                                        'WR-PR':'WR','RG':'G','RB-WR':'RB','FB-LB':'FB',
                                                        'LB-DE':'LB','RT':'T','DE-LB':'DE','DT-DE':'DT',
                                                        'NT-DE':'DT','C-G':'C','HB':'RB','LG':'G',
                                                        'T-G':'T','DE-DT':'DE','G-T':'G','G-C':'G',
                                                        'NT':'DT', 'EDGE':'DE', 'OG':'G',
                                                        'WR':'WR','LB':'LB','RB':'RB','DB':'DB','CB':'CB',
                                                        'DE':'DE','TE':'TE','DT':'DT','T':'T','QB':'QB',
                                                        'OLB':'OLB','G':'G','S':'S','C':'C','OT':'OT',
                                                        'DL':'DL','OL':'OL','K':'K','P':'P','ILB':'ILB',
                                                        'FS':'FS','SS':'SS','FB':'FB','LS':'LS'})
    
    return players_curr
    
    

In [3]:
#function to scrape/update defense weekly stats
def def_weekly_stats_scrape(first_yr, last_yr, nfl_sched):
    
    first_wk = 1
    num_years = last_yr - first_yr
    temp_df = pd.DataFrame()
    curr_yr = first_yr
    for i in range(0,num_years+1):
        if(curr_yr < 2021):
            weeks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
            year = [curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,
                   curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,
                   curr_yr,curr_yr,curr_yr,curr_yr,curr_yr]
        else:
            weeks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
            year = [curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,
                   curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,
                   curr_yr,curr_yr,curr_yr,curr_yr,curr_yr, curr_yr]
        

        curr_yr = curr_yr+1
        
        temp2 = pd.DataFrame(data = {'week':weeks,
                            'year':year})
        
    
        temp_df = temp_df.append(temp2)
    
    temp_df['page'] = 'https://fftoday.com/stats/playerstats.php?Season=' + temp_df['year'].astype(str) + '&GameWeek=' + temp_df['week'].astype(str) + '&PosID=99&LeagueID=26955'
    temp_df['page'] = temp_df['page'].astype(str)
    temp_df = temp_df.reset_index()
    temp_df = temp_df.drop(columns = ['index'])
    df2 = pd.DataFrame()

    for j in range(len(temp_df)):
        url = temp_df['page'][j]
        data = requests.get(url).text
        soup = BeautifulSoup(data, 'html.parser')
        dfs = pd.read_html(data)
        df = dfs[7]
        df = df.drop(labels = [0,1], axis = 0)
        df.columns = ['Team','G','Sack','FR', 'INT',
                   'DefTD','PA','PaYd/G','RuYd/G',
                 'Safety','KickTD','FPts','FPts/G']
        df['week_num'] = temp_df['week'][j]
        df['year'] = temp_df['year'][j]
        df2 = df2.append(df)
    
    new = df2['Team'].str.split(" ", expand = True)
    df2['Team'] = new[1]
    
    df2['abbr'] = df2['Team']
    df2['abbr'] = df2['abbr'].map({'Cardinals':'ARI','49ers':'SF','Saints':'NO',
                                   'Panthers':'CAR','Cowboys':'DAL','Texans':'HOU',
                                   'Raiders':'LV','Rams':'LAR','Lions':'DET',
                                   'Chiefs':'KC','Commanders':'WSH','Ravens':'BAL',
                                   'Colts':'IND','Steelers':'PIT','Bengals':'CIN',
                                   'Seahawks':'SEA','Dolphins':'MIA','Vikings':'MIN',
                                   'Patriots':'NE','Giants':'NYG','Titans':'TEN',
                                   'Broncos':'DEN', 'Jets':'NYJ','Eagles':'PHI',
                                   'Chargers':'LAC','Buccaneers':'TB','Bills':'BUF',
                                   'Browns':'CLE','Falcons':'ATL','Jaguars':'JAX',
                                   'Bears':'CHI','Packers':'GB'})
    
    df2 = df2.rename(columns = {'abbr':'CITY',
                               'PaYd/G':'PaYD',
                               'RuYd/G':'RuYd'})
    df2['Week'] = df2['week_num']
    df2['team_wk_year'] = df2['CITY'] + " " + df2['week_num'].astype(str) + " " + df2['year'].astype(str)
    nfl_sched['team_wk_year'] = nfl_sched['TEAM'] + " " + nfl_sched['Week'].astype(str) + " " + nfl_sched['Year'].astype(str)
    nfl_sched = nfl_sched.rename(columns = {'Opponent':'Opp'})
    nfl_sched = nfl_sched[['team_wk_year', 'Opp']]
    nfl_sched['away flag'] = np.where(nfl_sched['Opp'].astype(str).str[0] == '@', 1, 0)
    
    df2 = pd.merge(df2, nfl_sched, how = 'inner', on = 'team_wk_year', 
                  suffixes = (None, None))
    
    df2 = df2[['Team','CITY','Week','Sack','FR','INT','DefTD',
                'PA','PaYD','RuYd','Safety',
                'KickTD','FPts','week_num','year',
                'Opp','away flag','team_wk_year']]
    return df2
    

In [1]:
def position_feat_eng_df_creation(player_df,
                                  def_df,
                                  position,
                                 file_save):
    
    #if (position == 'QB'):
        qb = player_df[player_df['Position'] == position]
        qb = qb[qb['Player'] != 'Zach Miller']
        qb = qb.reset_index()
        qb = qb.drop(columns = ['index'])
        
        df_qb = pd.DataFrame()
        #for i in range(0,1):
        print('test')
        for i in range(len(qb)):
            time.sleep(3.1)
            print(i)
            player = qb['Player'][i]
            print(qb['Player'][i])
            url = qb['Gamelog'][i]
            #print(qb['Player'][i])
            data = requests.get(url).text
            #print(data)
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup)
            dfs = pd.read_html(data)
            df = dfs[0]
            df['Player'] = player
            #print(df.head())    
            #print(df.columns)
            df_qb = pd.concat([df_qb, df], axis=0, ignore_index=True)
        
        df_qb.columns = df_qb.columns.get_level_values(0) + '' +  df_qb.columns.get_level_values(1)
        if (position == 'QB'):
            print(df_qb.columns)

            df_qb = df_qb.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                       'Kick ReturnsRt','Kick ReturnsTD',
                                       'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                       'Punt ReturnsRet', 'Punt ReturnsTD',
                                       'Punt ReturnsY/R','Punt ReturnsYds',
                                       'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                       'PuntingYds', 'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL','Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                       'Unnamed: 35_level_0Sk',# 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status','Unnamed: 53_level_0Status', 'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status'])
            print(df_qb.columns)
        
            df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds', 'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                        'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                        'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                        'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                        'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                        'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                        'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            print(df_qb.columns)

        elif (position == 'RB'):
            df_qb = df_qb.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                       #'Punt ReturnsRet', 'Punt ReturnsTD',
                                       #'Punt ReturnsY/R','Punt ReturnsYds',
                                      #'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                      #'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                     #  'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk',# 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', #'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status',
                                             'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status'])
            print(df_qb.columns)
            df_qb = df_qb.drop(columns = ['Def InterceptionsInt', 'Def InterceptionsPD', 'Def InterceptionsTD',
               'Def InterceptionsYds',  'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
               'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk','Unnamed: 19_level_0Sk',
               'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
               'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
               'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
               'Unnamed: 25_level_0Status', 'Unnamed: 27_level_0Status',
                'Unnamed: 30_level_0Sk','Unnamed: 30_level_0Status', 'Unnamed: 31_level_0Status',
               'Unnamed: 32_level_0Sk', 'Unnamed: 33_level_0Status',
               'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Status',
               'Unnamed: 39_level_0Sk', 'Unnamed: 39_level_0Status',
               'Unnamed: 40_level_0Sk',
               'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
               'Unnamed: 44_level_0Sk', 'Unnamed: 45_level_0Status',
               'Unnamed: 50_level_0Status',
               'Unnamed: 52_level_0Status', 'Unnamed: 55_level_0Status',
               'Unnamed: 56_level_0Status', 'Unnamed: 57_level_0Status',
               'Unnamed: 58_level_0Status',
               'Unnamed: 62_level_0Status','Unnamed: 53_level_0Status','Unnamed: 35_level_0Sk'])
            print(df_qb.columns)

            df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            print(df_qb.columns)
        
        elif (position == 'WR'):
            print(df_qb.columns)
            df_qb = df_qb.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct','Def InterceptionsInt',
                                           'Def InterceptionsPD', 'Def InterceptionsTD',
                                   'Def InterceptionsYds',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                      # 'Punt ReturnsRet', 'Punt ReturnsTD',
                                      # 'Punt ReturnsY/R','Punt ReturnsYds',
                                    #   'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                     #  'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                      # 'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', #'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status',# 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
                                           'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status','Unnamed: 59_level_0Status', 'Unnamed: 60_level_0Status',
                               'Unnamed: 62_level_0Status', 'Unnamed: 65_level_0Status',
                               'Unnamed: 66_level_0Status','ScoringXP%', 'ScoringXPA', 'ScoringXPM',
                                           'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
                                   'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk','Unnamed: 19_level_0Sk',
                                          'Unnamed: 20_level_0Sk', 'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
                                   'Unnamed: 23_level_0Sk', 'Unnamed: 23_level_0Status',
                                   'Unnamed: 24_level_0Sk', 'Unnamed: 24_level_0Status',
                                   'Unnamed: 25_level_0Sk', 'Unnamed: 25_level_0Status',
                                       'Unnamed: 26_level_0Status', 'Unnamed: 27_level_0Status',
                                           'Unnamed: 29_level_0Status','Unnamed: 30_level_0Sk',
                                   'Unnamed: 30_level_0Status', 'Unnamed: 31_level_0Sk',
                                   'Unnamed: 31_level_0Status', 'Unnamed: 32_level_0Sk',
                                   'Unnamed: 33_level_0Status', 'Unnamed: 35_level_0Status',
                                           'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Status',
                                       'Unnamed: 39_level_0Sk', 'Unnamed: 39_level_0Status','Unnamed: 40_level_0Sk', 'Unnamed: 41_level_0Sk',
                                   'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
                                   'Unnamed: 44_level_0Sk', 'Unnamed: 45_level_0Status',
                                   'Unnamed: 49_level_0Status','Unnamed: 50_level_0Status',
                                   'Unnamed: 52_level_0Status', 'Unnamed: 55_level_0Status',
                                   'Unnamed: 56_level_0Status', 'Unnamed: 57_level_0Status',
                                   'Unnamed: 58_level_0Status','ScoringFG%', 'ScoringFGA', 'ScoringFGM'
                                          ])


            print(df_qb.columns)
        
            df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            print(df_qb.columns)
         
        elif (position == 'TE'):
            print(df_qb.columns)
            df_qb = df_qb.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                            'Def InterceptionsInt', 
                                           'Def InterceptionsPD', 'Def InterceptionsTD',
                                           'Def InterceptionsYds', 
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                      # 'Punt ReturnsRet', 'Punt ReturnsTD',
                                      # 'Punt ReturnsY/R','Punt ReturnsYds',
                                    #   'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                     #  'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                    #   'Unnamed: 26_level_0Sk', 
                                           'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                      # 'Unnamed: 33_level_0Sk', 
                                           'Unnamed: 34_level_0Status',
                                      # 'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', #'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status',#'Unnamed: 43_level_0Sk',
                                     #      'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status',# 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
                                           'Unnamed: 54_level_0Status',
                                            'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
                                           'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk',
                                           'Unnamed: 19_level_0Sk','Unnamed: 20_level_0Sk',
                                           'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
                                           'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
                                           'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
                                           'Unnamed: 26_level_0Status', 'Unnamed: 27_level_0Status',
                                           'Unnamed: 29_level_0Status', 'Unnamed: 30_level_0Status',
                                           'Unnamed: 31_level_0Status', 'Unnamed: 32_level_0Sk',
                                           'Unnamed: 33_level_0Status', 'Unnamed: 35_level_0Sk',
                                           'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Sk',
                                           'Unnamed: 37_level_0Status', 'Unnamed: 39_level_0Status',
                                           'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
                                           'Unnamed: 45_level_0Status',
                                            'Unnamed: 50_level_0Status',
                                           'Unnamed: 55_level_0Status', 'Unnamed: 58_level_0Status'
                               # 'Unnamed: 61_level_0Status'
                                          ])

            print(df_qb.columns)
        
            df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
         
            print(df_qb.columns)
        
        elif (position == 'K'):
            print(df_qb.columns)
            
            df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
                      'FumblesFL':'Fumbles FL',
                      'FumblesFR':'Fumbles FR',
                      'FumblesFmb':'Fumbles Fmb',
                      'FumblesTD':'Fumbles TD',
                            'FumblesYds':'Fumbles Yds',
                                   'Kick ReturnsRt':'Kick Returns Rt',
                                'Kick ReturnsTD':'Kick Returns TD',
                           'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                'Kick ReturnsYds':'Kick Returns Yds',
                                'Off. SnapsNum':'Off. Snaps Num',
                            'Off. SnapsPct':'Off. Snaps Pct',
                            'Player':'Player ', 
                                'Punt ReturnsRet':'Punt Returns Ret',
                                'Punt ReturnsTD':'Punt Returns TD',
                           'Punt ReturnsY/R':'Punt Returns Y/R',
                                'Punt ReturnsYds':'Punt Returns Yds',
                                'ReceivingCtch%':'Receiving Ctch%',
                                'ReceivingRec':'Receiving Rec',
                                'ReceivingTD':'Receiving TD',
                           'ReceivingTgt':'Receiving Tgt',
                                'ReceivingY/R':'Receiving Y/R',
                                'ReceivingY/Tgt':'Receiving Y/Tgt',
                                'ReceivingYds':'Receiving Yds',
                                'RushingAtt':'Rushing Att',
                            'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                            'RushingYds':'Rushing Yds',
                            'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                            'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                            'ScoringTD':'Scoring TD',
                            'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                            'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                            'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                            'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                            'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                      'Unnamed: 8_level_0Opp':'Opp',
                            'Unnamed: 9_level_0Result':'Result'})
            
            df_qb = df_qb[['Player ','ST Snaps Num', 'ST Snaps Pct', 'Scoring Pts',
                           'ScoringXPM','ScoringXPA','ScoringXP%','ScoringFGM','ScoringFGA',
                           'ScoringFG%','ScoringTD',
                          'Rk', 'GS', 'Year', 'Date', 'G#', 'Week', 'Age', 'Tm',
                          'home_away', 'Opp','Result']]
    

#             df_qb = df_qb.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
#                                            # 'Def InterceptionsInt', 
#                                            #'Def InterceptionsPD', 'Def InterceptionsTD',
#                                            #'Def InterceptionsYds', 
#                                        #'Kick ReturnsRt','Kick ReturnsTD',
#                                        #'Kick ReturnsY/Rt', 'Kick ReturnsYds',
#                                        #'Punt ReturnsRet', 'Punt ReturnsTD',
#                                        #'Punt ReturnsY/R','Punt ReturnsYds',
#                                        'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
#                                       'PuntingYds', 
#                                         'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
#                                        'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
#                                        'TacklesAst', 'TacklesComb',
#                                        'TacklesQBHits', 'TacklesSolo',
#                                        'TacklesTFL',
#                                              'PassingAY/A','PassingAtt',
#                                         'PassingCmp', 'PassingCmp%',
#                                         'PassingInt', 'PassingRate',
#                                         'PassingSk', 'PassingTD',
#                                         'PassingY/A', 'PassingYds',
#                                         'PassingYds.1',
#                                        #    'Unnamed: 21_level_0Status',
#                                       # 'Unnamed: 26_level_0Sk', 
#                                        #    'Unnamed: 28_level_0Sk',
#                                        #'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
#                                        #'Unnamed: 32_level_0Status',
#                                       # 'Unnamed: 33_level_0Sk', 
#                                         #   'Unnamed: 34_level_0Status',
#                                       # 'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
#                                        #'Unnamed: 36_level_0Sk', #'Unnamed: 37_level_0Sk',
#                                        #'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
#                                        #'Unnamed: 41_level_0Status',#'Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
#                                        'Unnamed: 46_level_0Status',# 'Unnamed: 47_level_0Status',
#                                        'Unnamed: 48_level_0Status',# 'Unnamed: 49_level_0Status',
#                              #   'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
#                                            'Unnamed: 54_level_0Status',
#                                             'Unnamed: 11_level_0Sk',# 'Unnamed: 15_level_0Sk',
#                                            'Unnamed: 17_level_0Status',# 'Unnamed: 18_level_0Sk',
#                                            'Unnamed: 19_level_0Sk',#'Unnamed: 20_level_0Sk',
#                                            #'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
#                                            'Unnamed: 23_level_0Status', #'Unnamed: 24_level_0Sk',
#                                           # 'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
#                                           # 'Unnamed: 26_level_0Status', 'Unnamed: 27_level_0Status',
#                                           # 'Unnamed: 29_level_0Status', 'Unnamed: 30_level_0Status',
#                                            'Unnamed: 31_level_0Status', #'Unnamed: 32_level_0Sk',
#                                           # 'Unnamed: 33_level_0Status', 'Unnamed: 35_level_0Sk',
#                                            'Unnamed: 36_level_0Status',# 'Unnamed: 37_level_0Sk',
#                                            'Unnamed: 37_level_0Status',# 'Unnamed: 39_level_0Status',
#                                            'Unnamed: 42_level_0Status',# 'Unnamed: 43_level_0Status',
#                                      #      'Unnamed: 45_level_0Status'#,
#                                          #   'Unnamed: 50_level_0Status',
#                                           # 'Unnamed: 55_level_0Status', 'Unnamed: 58_level_0Status'
#                                # 'Unnamed: 61_level_0Status'
#                                           ])

            print(df_qb.columns)
        
#             df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
#                                   'FumblesFL':'Fumbles FL',
#                                   'FumblesFR':'Fumbles FR',
#                                   'FumblesFmb':'Fumbles Fmb',
#                                   'FumblesTD':'Fumbles TD',
#                                         'FumblesYds':'Fumbles Yds',
#                                                'Kick ReturnsRt':'Kick Returns Rt',
#                                             'Kick ReturnsTD':'Kick Returns TD',
#                                        'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
#                                             'Kick ReturnsYds':'Kick Returns Yds',
#                                             'Off. SnapsNum':'Off. Snaps Num',
#                                         'Off. SnapsPct':'Off. Snaps Pct',
#                                        # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
#                                        # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
#                                        # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
#                                        # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
#                                        # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
#                                        # 'PassingYds.1':'Passing Yds.1',
#                                         'Player':'Player ', 
#                                             'Punt ReturnsRet':'Punt Returns Ret',
#                                             'Punt ReturnsTD':'Punt Returns TD',
#                                        'Punt ReturnsY/R':'Punt Returns Y/R',
#                                             'Punt ReturnsYds':'Punt Returns Yds',
                                               
#                                             'ReceivingCtch%':'Receiving Ctch%',
#                                             'ReceivingRec':'Receiving Rec',
#                                             'ReceivingTD':'Receiving TD',
#                                        'ReceivingTgt':'Receiving Tgt',
#                                             'ReceivingY/R':'Receiving Y/R',
#                                             'ReceivingY/Tgt':'Receiving Y/Tgt',
#                                             'ReceivingYds':'Receiving Yds',
                                            
#                                             'RushingAtt':'Rushing Att',
#                                         'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
#                                         'RushingYds':'Rushing Yds',
#                                         'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
#                                         'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
#                                         'ScoringTD':'Scoring TD',
                                            
#                                      #   'TacklesAst':'Tackles Ast',
#                                       #      'TacklesComb':'Tackles Comb',
#                                       # 'TacklesQBHits':'Tackles QBHits',
#                                       #      'TacklesSolo':'Tackles Solo',
#                                       # 'TacklesTFL':'TacklesTFL',
                                            
#                                         'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
#                                         'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
#                                         'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
#                                         'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
#                                         'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
#                                   'Unnamed: 8_level_0Opp':'Opp',
#                                         'Unnamed: 9_level_0Result':'Result'})
    
        
        
        print(df_qb.shape)
        df_qb2 = df_qb[df_qb['G#'] != '']
        df_qb2 = df_qb2[df_qb2['G#'] != 'G#']
        df_qb2 = df_qb2[~df_qb2['Date'].str.contains("Game")]
   #     df_qb2 = df_qb2[df_qb2['GS'] != 'Did Not Play']
        
        df_qb2 = df_qb2[~df_qb2['GS'].isin(['Did Not Play', 'Inactive',
                                           'Injured Reserve','Non-Football Injury',
                                           'Suspended','Returned from Injured Reserve',
                                           'COVID-19 List','Exempt List',
                                           'Physically Unable to Perform'])]

    
        df_qb2['player_wk_year'] = df_qb2['Player '] + " " + df_qb2['Week'].astype(str) + " " + df_qb2['Year'].astype(str)
        print('yay')
        
       #return df_qb2
        
        test_df = df_qb2[['Player ','Week', 'Year', 'player_wk_year']]
        test_df['Week'] = test_df['Week'].astype(int)
        test_df['Year'] = test_df['Year'].astype(int)     
        print(test_df.shape)
        for i in range(1,36):
            print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            player_wks_ago = "player " + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "Week"
                curr_yr = "Year"
                test_df[curr_wk] = test_df['Week']
                test_df[curr_yr] = test_df['Year']
               # test_df[curr_wk] = test_df[curr_wk].astype(int)
               # test_df[curr_yr] = test_df[curr_yr].astype(int)
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])

            test_df[player_wks_ago] = test_df['Player '] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

            
        print(test_df.shape)
        new_df = pd.DataFrame()
        for i in range(0,36):
            print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df2 = test_df.drop(columns = ['Player ', 'Week', 'Year'])
                new_df = pd.merge(test_df2, df_qb2, how = 'left', on = ['player_wk_year'], suffixes = (None, None))
                print(new_df.head())
            else:
                temp_df = df_qb2.drop(columns = ['Rk','GS','Year','G#'])
                for col in temp_df.columns:
                    new_col = col + " " + str(i) + " wks ago"
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                merge_column = "player " + str(i) + " wks ago"
                temp_df[merge_column] = df_qb2['player_wk_year']
                new_df = pd.merge(new_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
        
        qb_df = new_df
        print(qb_df.shape)
        tms = ['Tm','Tm 1 wks ago', 'Tm 2 wks ago',
              'Tm 3 wks ago','Tm 4 wks ago','Tm 5 wks ago','Tm 6 wks ago',
              'Tm 7 wks ago','Tm 8 wks ago','Tm 9 wks ago','Tm 10 wks ago',
              'Tm 11 wks ago','Tm 12 wks ago','Tm 13 wks ago','Tm 14 wks ago',
              'Tm 15 wks ago','Tm 16 wks ago','Tm 17 wks ago','Tm 18 wks ago',
              'Tm 19 wks ago','Tm 20 wks ago','Tm 21 wks ago','Tm 22 wks ago',
              'Tm 23 wks ago','Tm 24 wks ago','Tm 25 wks ago','Tm 26 wks ago',
              'Tm 27 wks ago','Tm 28 wks ago','Tm 29 wks ago','Tm 30 wks ago',
              'Tm 31 wks ago','Tm 32 wks ago','Tm 33 wks ago','Tm 34 wks ago',
              'Tm 35 wks ago']

        opp = ['Opp','Opp 1 wks ago', 'Opp 2 wks ago',
              'Opp 3 wks ago','Opp 4 wks ago','Opp 5 wks ago','Opp 6 wks ago',
              'Opp 7 wks ago','Opp 8 wks ago','Opp 9 wks ago','Opp 10 wks ago',
              'Opp 11 wks ago','Opp 12 wks ago','Opp 13 wks ago','Opp 14 wks ago',
              'Opp 15 wks ago','Opp 16 wks ago','Opp 17 wks ago','Opp 18 wks ago',
              'Opp 19 wks ago','Opp 20 wks ago','Opp 21 wks ago','Opp 22 wks ago',
              'Opp 23 wks ago','Opp 24 wks ago','Opp 25 wks ago','Opp 26 wks ago',
              'Opp 27 wks ago','Opp 28 wks ago','Opp 29 wks ago','Opp 30 wks ago',
              'Opp 31 wks ago','Opp 32 wks ago','Opp 33 wks ago','Opp 34 wks ago',
              'Opp 35 wks ago']
        
        for i in tms:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE',
                               'NOR': 'NO', 'SFO': 'SF', 'SDG':'LAC',
                              'GNB':'GB','TAM':'TB','WAS':'WSH',
                              'KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV', 'IND':'IND', 'SEA':'SEA',
                             'HOU':'HOU', 'ATL':'ATL', 'CIN':'CIN',
                             'PIT':'PIT','NYG':'NYG', 'DET':'DET',
                             'CLE':'CLE','DEN':'DEN','DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI',
                             'BAL':'BAL','JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA',
                             'CAR':'CAR','PHI':'PHI','MIN':'MIN',
                             'TEN':'TEN', 'LAR':'LAR', 'LAC':'LAC'})
        for i in opp:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE', 'NOR': 'NO',
                              'SFO': 'SF', 'SDG':'LAC', 'GNB':'GB','TAM':'TB',
                              'WAS':'WSH','KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV','IND':'IND', 'SEA':'SEA', 'HOU':'HOU',
                             'ATL':'ATL','CIN':'CIN', 'PIT':'PIT', 'NYG':'NYG',
                             'DET':'DET','CLE':'CLE', 'DEN':'DEN', 'DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI','BAL':'BAL',
                             'JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA', 'CAR':'CAR',
                             'PHI':'PHI', 'MIN':'MIN', 'TEN':'TEN', 'LAR':'LAR',
                             'LAC':'LAC'})    
            
        #opp
        opp_wk_year = ['opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']

        wk = ['Week', 'Week 1 wks ago', 'Week 2 wks ago', 'Week 3 wks ago', 'Week 4 wks ago', 'Week 5 wks ago',
              'Week 6 wks ago', 'Week 7 wks ago', 'Week 8 wks ago', 'Week 9 wks ago', 'Week 10 wks ago',
              'Week 11 wks ago', 'Week 12 wks ago', 'Week 13 wks ago', 'Week 14 wks ago', 'Week 15 wks ago',
              'Week 16 wks ago', 'Week 17 wks ago', 'Week 18 wks ago', 'Week 19 wks ago', 'Week 20 wks ago',
              'Week 21 wks ago', 'Week 22 wks ago', 'Week 23 wks ago', 'Week 24 wks ago', 'Week 25 wks ago',
              'Week 26 wks ago', 'Week 27 wks ago', 'Week 28 wks ago', 'Week 29 wks ago', 'Week 30 wks ago',
              'Week 31 wks ago', 'Week 32 wks ago', 'Week 33 wks ago', 'Week 34 wks ago', 'Week 35 wks ago',
              'Week 36 wks ago']

        year = ['Year', 'year 1 wks ago', 'year 2 wks ago', 'year 3 wks ago', 'year 4 wks ago', 'year 5 wks ago',
               'year 6 wks ago', 'year 7 wks ago', 'year 8 wks ago', 'year 9 wks ago', 'year 10 wks ago',
               'year 11 wks ago', 'year 12 wks ago', 'year 13 wks ago', 'year 14 wks ago', 'year 15 wks ago',
               'year 16 wks ago', 'year 17 wks ago', 'year 18 wks ago', 'year 19 wks ago', 'year 20 wks ago',
               'year 21 wks ago', 'year 22 wks ago', 'year 23 wks ago', 'year 24 wks ago', 'year 25 wks ago',
               'year 26 wks ago', 'year 27 wks ago', 'year 28 wks ago', 'year 29 wks ago', 'year 30 wks ago',
               'year 31 wks ago', 'year 32 wks ago', 'year 33 wks ago', 'year 34 wks ago', 'year 35 wks ago',
               'year 36 wks ago']
        
        
        for i in range(0,36):
            #print(i)
            #print(opp[i])
            #print(wk[i])
            #print(year[i])
            opp_colname = opp[i]
            wk_colname = wk[i]
            year_colname = year[i]
            opp_wk_year_colname = opp_wk_year[i]
            if (i== 0):
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)
            else:
                #qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str).str[:-2] + " " + qb_df[year_colname].astype(str)
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)
        
        #if (i == 0):

        test_df = qb_df[['player_wk_year','opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']]
        print(test_df.shape)
        new_df = pd.DataFrame()

        for i in range(0,36):
            #print(i)
            if (i == 0):
                #new_df = df_qb2
                #test_df2 = test_df.drop(columns = [])
                new_df = pd.merge(test_df, def_df, how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
                #print(new_df.head())
            else:
                temp_df = def_df.drop(columns = ['Week','week_num','year','Opp','team_wk_year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'opp' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                temp_df['team_wk_year'] = def_df['team_wk_year']
               # print(temp_df2.head())
                qb_no_null = test_df[[opp_wk_year[i]]]
               # print(qb_no_null.head())
                qb_no_null = qb_no_null[qb_no_null.notnull()]
              #  print(qb_no_null.head())
                qb_no_null = pd.merge(qb_no_null, temp_df, how = 'inner', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None,None))
                #print(qb_no_null.head())
                qb_no_null = qb_no_null.drop_duplicates()
        
                qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                new_df = pd.merge(new_df, qb_no_null, how = 'left', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
        
        new_df = new_df.drop_duplicates()
        
        qb_df2 = qb_df.drop(columns = ['Week', 'Opp', 'opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago',
           'opp 3 wk year ago', 'opp 4 wk year ago', 'opp 5 wk year ago',
           'opp 6 wk year ago', 'opp 7 wk year ago', 'opp 8 wk year ago',
           'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
           'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago',
           'opp 15 wk year ago', 'opp 16 wk year ago', 'opp 17 wk year ago',
           'opp 18 wk year ago', 'opp 19 wk year ago', 'opp 20 wk year ago',
           'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
           'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago',
           'opp 27 wk year ago', 'opp 28 wk year ago', 'opp 29 wk year ago',
           'opp 30 wk year ago', 'opp 31 wk year ago', 'opp 32 wk year ago',
           'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago'])
        

        new_df2 = pd.merge(qb_df2,new_df, how = 'left', on = 'player_wk_year', suffixes = (None,None))
        print(new_df2.shape)
        test_df = def_df[['CITY','week_num', 'year', 'team_wk_year']]
        for i in range(1,36):
            #print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            #player_wks_ago = "player " + str(i) + " wks ago"
            team_wk_year = 'team_wk_year ' + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "week_num"
                curr_yr = "year"
                test_df[curr_wk] = test_df['week_num']
                test_df[curr_yr] = test_df['year']
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk] - 1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk]-1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr] -1, test_df[curr_yr])

            test_df[team_wk_year] = test_df['CITY'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)
        
        return new_df2
            
#         #test_df.head()
#         new_def_df = pd.DataFrame()
#         return test_df
#         for i in range(0,36):
#             #print(i)
#             if (i == 0):
#                 #new_df = df_qb2
#                 test_df = test_df.drop(columns = ['CITY','week_num','year'])
#                 new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['team_wk_year'], suffixes = (None, None))
#               #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
#                 new_def_df = new_def_df.drop_duplicates(subset=['team_wk_year'], keep='last')
                
#                 print(new_def_df.head())
#             else:
#                 temp_df = def_df.drop(columns = ['Team','CITY','Week','week_num','year'])
#               #  temp_df2 = temp_df
#                 for col in temp_df.columns:
#                     new_col = 'def matchup' + " " + col + " " + str(i) + 'wks ago'
#                     temp_df[new_col] = temp_df[col]
#                     temp_df = temp_df.drop(columns = [col])
#                 #merge_column = "player " + str(i) + " wks ago"
#                 #merge_column = 
#                 merge_column = "team_wk_year " + str(i) + " wks ago"
#                 temp_df[merge_column] = def_df['team_wk_year']
#                 #temp_df['team_wk_year'] = def_df['team_wk_year']
#                # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
#                 temp_df = temp_df.drop_duplicates(subset=[merge_column], keep='last')
#                 temp_df = temp_df.drop_duplicates()
#                 new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
#                 new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
#         new_def_df2 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
#                'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
#                'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
#                'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
#                'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
#                'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
#                'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
#                'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
#                'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
#                'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
#                'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
#                'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
#                'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
#                'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
#                'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
#                'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
#                'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
#                'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
#                'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
#                'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
#                'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
#                'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
#                'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
#                'year 35 wks ago', 'Team', 'CITY', 'Week', 'Sack', 'FR', 'INT', 'DefTD',
#                'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD', 'FPts', 'week_num',
#                'year', 'Opp', 'away flag'#, 'opp_wk_year'
#                                                 ])
        
#         new_df3 = pd.merge(new_df2,new_def_df2,how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None,None))
#         print(new_df3.shape)
#         new_df3.to_csv(file_save)
#         return new_df3

In [1]:
def position_feat_eng_df_creation2(new_df2, def_df, position,
                                  file_save):
        test_df = def_df[['CITY','week_num', 'year', 'team_wk_year']]
        for i in range(1,36):
            #print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            #player_wks_ago = "player " + str(i) + " wks ago"
            team_wk_year = 'team_wk_year ' + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "week_num"
                curr_yr = "year"
                test_df[curr_wk] = test_df['week_num']
                test_df[curr_yr] = test_df['year']
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk] - 1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk]-1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr] -1, test_df[curr_yr])

            test_df[team_wk_year] = test_df['CITY'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)
        
   #     return new_df2
            
        #test_df.head()
        new_def_df = pd.DataFrame()
      #  return test_df
        for i in range(0,36):
            #print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df = test_df.drop(columns = ['CITY','week_num','year'])
                new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['team_wk_year'], suffixes = (None, None))
              #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
                new_def_df = new_def_df.drop_duplicates(subset=['team_wk_year'], keep='last')
                
                print(new_def_df.head())
            else:
                temp_df = def_df.drop(columns = ['Team','CITY','Week','week_num','year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'def matchup' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                merge_column = "team_wk_year " + str(i) + " wks ago"
                temp_df[merge_column] = def_df['team_wk_year']
                #temp_df['team_wk_year'] = def_df['team_wk_year']
               # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                temp_df = temp_df.drop_duplicates(subset=[merge_column], keep='last')
                temp_df = temp_df.drop_duplicates()
                new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
               # new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
        new_def_df2 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
               'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
               'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
               'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
               'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
               'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
               'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
               'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
               'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
               'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
               'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
               'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
               'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
               'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
               'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
               'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
               'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
               'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
               'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
               'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
               'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
               'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
               'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
               'year 35 wks ago', 'Team', 'CITY', 'Week', 'Sack', 'FR', 'INT', 'DefTD',
               'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD', 'FPts', 'week_num',
               'year', 'Opp', 'away flag', 'opp_wk_year'
                                                ])
        
        new_df3 = pd.merge(new_df2,new_def_df2,how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None,None))
        print(new_df3.shape)
        new_df3.to_csv(file_save)
        return new_df3

In [17]:
def create_qb_trend_var(qb_df, position, file_save):
    qb_df = qb_df.drop(columns = [#'Unnamed: 0',
                                  'week num 1 wks ago',
        'year 1 wks ago','player 1 wks ago','week num 2 wks ago','year 2 wks ago',
        'player 2 wks ago','week num 3 wks ago','year 3 wks ago','player 3 wks ago',
        'week num 4 wks ago','year 4 wks ago','player 4 wks ago','week num 5 wks ago',
        'year 5 wks ago','player 5 wks ago','week num 6 wks ago','year 6 wks ago',
        'player 6 wks ago','week num 7 wks ago','year 7 wks ago','player 7 wks ago',
        'week num 8 wks ago','year 8 wks ago','player 8 wks ago','week num 9 wks ago',
        'year 9 wks ago','player 9 wks ago','week num 10 wks ago','year 10 wks ago',
        'player 10 wks ago','week num 11 wks ago','year 11 wks ago','player 11 wks ago',
        'week num 12 wks ago','year 12 wks ago','player 12 wks ago','week num 13 wks ago',
        'year 13 wks ago','player 13 wks ago','week num 14 wks ago','year 14 wks ago',
        'player 14 wks ago','week num 15 wks ago','year 15 wks ago','player 15 wks ago',
        'week num 16 wks ago','year 16 wks ago','player 16 wks ago','week num 17 wks ago',
        'year 17 wks ago','player 17 wks ago','week num 18 wks ago','year 18 wks ago',
        'player 18 wks ago','week num 19 wks ago','year 19 wks ago','player 19 wks ago',
        'week num 20 wks ago','year 20 wks ago','player 20 wks ago','week num 21 wks ago',
        'year 21 wks ago','player 21 wks ago','week num 22 wks ago','year 22 wks ago',
        'player 22 wks ago','week num 23 wks ago','year 23 wks ago','player 23 wks ago',
        'week num 24 wks ago','year 24 wks ago','player 24 wks ago','week num 25 wks ago',
        'year 25 wks ago','player 25 wks ago','week num 26 wks ago','year 26 wks ago',
        'player 26 wks ago','week num 27 wks ago','year 27 wks ago','player 27 wks ago',
        'week num 28 wks ago','year 28 wks ago','player 28 wks ago','week num 29 wks ago',
        'year 29 wks ago','player 29 wks ago','week num 30 wks ago','year 30 wks ago',
        'player 30 wks ago','week num 31 wks ago','year 31 wks ago','player 31 wks ago',
        'week num 32 wks ago','year 32 wks ago','player 32 wks ago','week num 33 wks ago',
        'year 33 wks ago','player 33 wks ago','week num 34 wks ago','year 34 wks ago',
        'player 34 wks ago','week num 35 wks ago','year 35 wks ago','player 35 wks ago',
        #'Unnamed: 0.1','Unnamed: 0.1.1',
        'opp 1 wk year ago','opp 2 wk year ago',
        'opp 3 wk year ago','opp 4 wk year ago','opp 5 wk year ago','opp 6 wk year ago',
        'opp 7 wk year ago','opp 8 wk year ago','opp 9 wk year ago','opp 10 wk year ago',
        'opp 11 wk year ago','opp 12 wk year ago','opp 13 wk year ago','opp 14 wk year ago',
        'opp 15 wk year ago','opp 16 wk year ago','opp 17 wk year ago','opp 18 wk year ago',
        'opp 19 wk year ago','opp 20 wk year ago','opp 21 wk year ago','opp 22 wk year ago',
        'opp 23 wk year ago','opp 24 wk year ago','opp 25 wk year ago','opp 26 wk year ago',
        'opp 27 wk year ago','opp 28 wk year ago','opp 29 wk year ago','opp 30 wk year ago',
        'opp 31 wk year ago','opp 32 wk year ago','opp 33 wk year ago','opp 34 wk year ago',
         'opp 35 wk year ago'
        ,'team_wk_year 1 wks ago','team_wk_year 2 wks ago','team_wk_year 3 wks ago',
         'team_wk_year 4 wks ago','team_wk_year 5 wks ago','team_wk_year 6 wks ago','team_wk_year 7 wks ago',
         'team_wk_year 8 wks ago','team_wk_year 9 wks ago','team_wk_year 10 wks ago','team_wk_year 11 wks ago',
         'team_wk_year 12 wks ago','team_wk_year 13 wks ago','team_wk_year 14 wks ago','team_wk_year 15 wks ago',
         'team_wk_year 16 wks ago','team_wk_year 17 wks ago','team_wk_year 18 wks ago','team_wk_year 19 wks ago',
         'team_wk_year 20 wks ago','team_wk_year 21 wks ago','team_wk_year 22 wks ago','team_wk_year 23 wks ago',
         'team_wk_year 24 wks ago','team_wk_year 25 wks ago','team_wk_year 26 wks ago','team_wk_year 27 wks ago',
         'team_wk_year 28 wks ago','team_wk_year 29 wks ago','team_wk_year 30 wks ago','team_wk_year 31 wks ago',
         'team_wk_year 32 wks ago','team_wk_year 33 wks ago','team_wk_year 34 wks ago','team_wk_year 35 wks ago'
        ])
    qb_df = qb_df.drop_duplicates()
    print(qb_df.shape)
    
    print('trend 1 creation')
    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = qb_df[['player_wk_year']]    
    temp_df2 = pd.DataFrame()     
    features = ['opp Sack', 'opp FR', 'opp INT', 'opp DefTD', 'opp PA', 'opp PaYD', 'opp RuYd',
       'opp Safety', 'opp KickTD', 'opp FPts', 'opp away flag']
    
    #for feat in features:
    #    qb_df[feat] = qb_df[feat].astype('float')
    
    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + "wks ago"
            qb_df[fillna_col] = qb_df[fillna_col].fillna(0)
            qb_df[fillna_col] = qb_df[fillna_col].astype('float')
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = qb_df[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     
    print('merge 1')
    qb_df = pd.merge(qb_df, temp_df, how = 'inner', on= 'player_wk_year', suffixes = (None,None))
    qb_df = qb_df.drop_duplicates()
    print(qb_df.shape)
    
    print('trend 2 creation')

    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = qb_df[['player_wk_year']]    
    temp_df2 = pd.DataFrame()     
    if (position == 'QB'):
        
        features = ['Fumbles FF','Fumbles FL','Fumbles FR','Fumbles Fmb',
        'Fumbles TD','Fumbles Yds','Off. Snaps Num','Off. Snaps Pct','Passing AY/A',
        'Passing Att','Passing Cmp','Passing Cmp%','Passing Int','Passing Rate',
        'Passing Sk','Passing TD','Passing Y/A','Passing Yds',
        'Rushing Att','Rushing TD','Rushing Y/A','Rushing Yds','ST Snaps Num',
        'ST Snaps Pct','Scoring 2PM','Scoring Pts','Scoring TD',
        'Age']
    elif(position == 'RB' or position == 'WR'):
        
        features = ['Fumbles FF','Fumbles FL','Fumbles FR','Fumbles Fmb',
        'Fumbles TD','Fumbles Yds','Kick Returns Rt','Kick Returns TD',
        'Kick Returns Y/Rt','Kick Returns Yds','Off. Snaps Num','Off. Snaps Pct',
        'Punt Returns Ret','Punt Returns TD','Punt Returns Y/R','Punt Returns Yds',
        'Receiving Ctch%','Receiving Rec','Receiving TD','Receiving Tgt',
        'Receiving Y/R','Receiving Y/Tgt','Receiving Yds','Rushing Att',
        'Rushing TD','Rushing Y/A','Rushing Yds','ST Snaps Num',
        'ST Snaps Pct','Scoring 2PM','Scoring Pts',#'Scoring Sfty',
        'Scoring TD','Age']
    
    elif(position == 'TE'):
        
        features = ['Fumbles FF','Fumbles FL','Fumbles FR','Fumbles Fmb',
        'Fumbles TD','Fumbles Yds','Off. Snaps Num','Off. Snaps Pct',
        'Receiving Ctch%','Receiving Rec','Receiving TD','Receiving Tgt',
        'Receiving Y/R','Receiving Y/Tgt','Receiving Yds','Rushing Att',
        'Rushing TD','Rushing Y/A','Rushing Yds','ST Snaps Num',
        'ST Snaps Pct','Scoring 2PM','Scoring Pts',#'Scoring Sfty',
        'Scoring TD','Age']    
    elif(position == 'K'):
        features = ['ScoringXPM', 'ScoringXPA', 'ScoringXP%',
                    'ScoringFGM', 'ScoringFGA', 'ScoringFG%', 'Scoring Pts','Age']        
        


        
    #for feat in features:
    #    qb_df[feat] = qb_df[feat].astype('float')

    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + " wks ago"
            qb_df[fillna_col] = qb_df[fillna_col].fillna(0)
            if (feat == 'Off. Snaps Pct' or feat == 'Receiving Ctch%' or
               feat == 'ST Snaps Pct' or 'Passing Cmp%'):
                qb_df[fillna_col] = qb_df[fillna_col].astype(str)
                qb_df[fillna_col] = qb_df[fillna_col].str.rstrip('%').astype('float')/100.0
            else:   
                #print('not feat')
                qb_df[fillna_col] = qb_df[fillna_col].astype('float')
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = qb_df[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     

    print('merge 2')
    qb_df = pd.merge(qb_df, temp_df, how = 'inner', on= 'player_wk_year', suffixes = (None,None))
    qb_df = qb_df.drop_duplicates()

    print(qb_df.shape)

    print('trend 3 creation')
    
    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = qb_df[['player_wk_year']]    
    temp_df2 = pd.DataFrame()     
    features = ['def matchup Sack','def matchup FR',
    'def matchup INT','def matchup DefTD',
    'def matchup PA','def matchup PaYD',
    'def matchup RuYd','def matchup Safety',
    'def matchup KickTD','def matchup FPts',
    #'def matchup Opp',
                'def matchup away flag']
    
   # for feat in features:
   #     qb_df[feat] = qb_df[feat].astype('float')

    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + "wks ago"
            qb_df[fillna_col] = qb_df[fillna_col].fillna(0)
            qb_df[fillna_col] = qb_df[fillna_col].astype('float')
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = qb_df[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     

    print('merge 3')
    qb_df = pd.merge(qb_df, temp_df, how = 'inner', on= 'player_wk_year', suffixes = (None,None))
    qb_df = qb_df.drop_duplicates()

    print(qb_df.shape)

    qb_df.to_csv(file_save)
    return qb_df

In [8]:
#now we need to get our current week players and get an oot df
#for the week we care about
def create_curr_wk_oot_qb(nfl_sched,
                          year,
                          week,
                          player_df,
                          position,
                          def_df,
                         save_file):
    
        nfl_sched = nfl_sched[(nfl_sched['Year'] == year) &
                             (nfl_sched['Week'] == week)]
        nfl_sched = nfl_sched.reset_index()
        nfl_sched = nfl_sched.drop(columns = ['index'])
    
        qb = player_df[player_df['Position'] == position]
        qb = qb[qb['Player'] != 'Malik Willis']
        qb = qb[qb['Player'] != 'Brandon Allen']

        qb = qb[qb['Playing_Prd_End'] ==  year]
        qb = qb.reset_index()
        qb = qb.drop(columns = ['index'])
        
        df_qb = pd.DataFrame()
        #for i in range(1000):
        for i in range(len(qb)):
            time.sleep(3.1)
            print(i)
            player = qb['Player'][i]
            print(qb['Player'][i])
            url = qb['Gamelog'][i]
            #print(qb['Player'][i])
            data = requests.get(url).text
            #print(data)
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup)
            dfs = pd.read_html(data)
            df = dfs[0]
            df['Player'] = player
            #print(df.head())    
            #print(df.columns)
            df_qb = pd.concat([df_qb, df], axis=0, ignore_index=True)
        
        df_qb.columns = df_qb.columns.get_level_values(0) + '' +  df_qb.columns.get_level_values(1)
        df_qb = df_qb[['Player','Unnamed: 6_level_0Tm']]
        df_qb = df_qb.rename(columns = {'Unnamed: 6_level_0Tm':'TEAM'})
        df_qb = df_qb.drop_duplicates()
        df_qb['TEAM'] = df_qb['TEAM'].map({'NWE': 'NE',
                               'NOR': 'NO', 'SFO': 'SF', 'SDG':'LAC',
                              'GNB':'GB','TAM':'TB','WAS':'WSH',
                              'KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV', 'IND':'IND', 'SEA':'SEA',
                             'HOU':'HOU', 'ATL':'ATL', 'CIN':'CIN',
                             'PIT':'PIT','NYG':'NYG', 'DET':'DET',
                             'CLE':'CLE','DEN':'DEN','DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI',
                             'BAL':'BAL','JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA',
                             'CAR':'CAR','PHI':'PHI','MIN':'MIN',
                             'TEN':'TEN', 'LAR':'LAR', 'LAC':'LAC'})
        
        df_qb = pd.merge(df_qb, nfl_sched, how = 'inner', on = 'TEAM', 
                        suffixes = (None, None))
        df_qb = df_qb.rename(columns = {'TEAM':'team',
                                       'Player':'Player ',
                                       'Opponent':'opp'})
        df_qb['opp'] = df_qb['opp'].str.replace('@',"")

        df_qb['player_wk_year'] = df_qb['Player '] + " " + df_qb['Week'].astype(str) + " " + df_qb['Year'].astype(str)
        df_qb['opp wk year'] = df_qb['opp'] + " " + df_qb['Week'].astype(str) + " " + df_qb['Year'].astype(str)
        
        df = df_qb[['Week','Year','Player ','team',
                      'opp','player_wk_year','opp wk year']]
        
        
        qb = player_df[player_df['Position'] == position]
        qb = qb[qb['Player'] != 'Malik Willis']
        
        qb = qb.reset_index()
        qb = qb.drop(columns = ['index'])
        
        df_qb2 = pd.DataFrame()
        #for i in range(1000):
        for i in range(len(qb)):
            time.sleep(3.1)

            print(i)
            player = qb['Player'][i]
            print(qb['Player'][i])
            url = qb['Gamelog'][i]
            #print(qb['Player'][i])
            data = requests.get(url).text
            #print(data)
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup)
            dfs = pd.read_html(data)
            df2 = dfs[0]
            df2['Player'] = player
            #print(df.head())    
            #print(df.columns)
            df_qb2 = pd.concat([df_qb2, df2], axis=0, ignore_index=True)
        
        df_qb2.columns = df_qb2.columns.get_level_values(0) + '' +  df_qb2.columns.get_level_values(1)
        if (position == 'QB'):
            print(df_qb2.columns)
            df_qb2 = df_qb2.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                       'Kick ReturnsRt','Kick ReturnsTD',
                                       'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                       'Punt ReturnsRet', 'Punt ReturnsTD',
                                       'Punt ReturnsY/R','Punt ReturnsYds',
                                       'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                       'PuntingYds', 'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL','Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                       'Unnamed: 35_level_0Sk', #'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status','Unnamed: 53_level_0Status', 'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status'])
            print(df_qb2.columns)
            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds', 'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                        'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                        'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                        'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                        'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                        'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                        'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            print(df_qb2.columns)
        elif (position == 'WR'):
            

            df_qb2 = df_qb2.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                      # 'Punt ReturnsRet', 'Punt ReturnsTD',
                                      # 'Punt ReturnsY/R','Punt ReturnsYds',
                                       #'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                       #'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                       #'Unnamed: 35_level_0Sk', 
                                             'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk',# 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
                                             'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status','Def InterceptionsInt', 'Def InterceptionsPD', 'Def InterceptionsTD',
                           'Def InterceptionsYds', 'ScoringFG%', 'ScoringFGA', 'ScoringFGM', 
                                              'ScoringXP%', 'ScoringXPA',
                               'ScoringXPM', 'Unnamed: 11_level_0Sk',
                               'Unnamed: 15_level_0Sk', 'Unnamed: 17_level_0Status',
                               'Unnamed: 18_level_0Sk', 'Unnamed: 19_level_0Sk',
                                              'Unnamed: 20_level_0Sk', 'Unnamed: 21_level_0Sk',
                               'Unnamed: 22_level_0Sk', 'Unnamed: 23_level_0Sk',
                               'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
                               'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
                               'Unnamed: 25_level_0Status', 'Unnamed: 26_level_0Status',
                               'Unnamed: 27_level_0Status', 'Unnamed: 29_level_0Status', 
                                             'Unnamed: 30_level_0Sk', 'Unnamed: 30_level_0Status',
                               'Unnamed: 31_level_0Sk', 'Unnamed: 31_level_0Status',
                               'Unnamed: 32_level_0Sk', 'Unnamed: 33_level_0Status',
                               'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Status',
                               'Unnamed: 39_level_0Sk', 'Unnamed: 39_level_0Status',
                                             'Unnamed: 40_level_0Sk', 'Unnamed: 41_level_0Sk',
                               'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
                               'Unnamed: 44_level_0Sk', 'Unnamed: 45_level_0Status',
                                 'Unnamed: 50_level_0Status', 'Unnamed: 52_level_0Status',
                               'Unnamed: 55_level_0Status', 'Unnamed: 56_level_0Status',
                               'Unnamed: 57_level_0Status', 'Unnamed: 58_level_0Status',
                               'Unnamed: 59_level_0Status',   'Unnamed: 60_level_0Status',
                               'Unnamed: 62_level_0Status', 'Unnamed: 65_level_0Status',
                               'Unnamed: 66_level_0Status'])
        

            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
                
        elif (position == 'TE'):
            
            print(df_qb2.columns)
            df_qb2 = df_qb2.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                      # 'Punt ReturnsRet', 'Punt ReturnsTD',
                                      # 'Punt ReturnsY/R','Punt ReturnsYds',
                                      # 'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                     #  'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                      # 'Unnamed: 26_level_0Sk', 
                                             'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                      # 'Unnamed: 33_level_0Sk', 
                                             'Unnamed: 34_level_0Status',
                                       'Unnamed: 35_level_0Sk',# 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status',#'Unnamed: 43_level_0Sk',
                                             #'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status',# 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
                                             'Unnamed: 54_level_0Status',
                                #'Unnamed: 61_level_0Status'
                                             'Def InterceptionsInt', 'Def InterceptionsPD', 'Def InterceptionsTD',
                                           'Def InterceptionsYds',  'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
                                           'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk',
                                       'Unnamed: 19_level_0Sk','Unnamed: 20_level_0Sk',
                                       'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
                                       'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
                                       'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
                                       'Unnamed: 26_level_0Status', 'Unnamed: 27_level_0Status',
                                       'Unnamed: 29_level_0Status','Unnamed: 30_level_0Status',
                                       'Unnamed: 31_level_0Status', 'Unnamed: 32_level_0Sk',
                                           'Unnamed: 33_level_0Status', 'Unnamed: 36_level_0Status',
                                       'Unnamed: 37_level_0Status', 'Unnamed: 39_level_0Status',
                                             'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
                                       'Unnamed: 45_level_0Status','Unnamed: 50_level_0Status',
                                       'Unnamed: 55_level_0Status', 'Unnamed: 58_level_0Status'
                                            ])


            print(df_qb2.columns)
            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
                
        
        elif (position == 'RB'):
            print(df_qb2.columns)
            df_qb2 = df_qb2.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                       #'Punt ReturnsRet', 'Punt ReturnsTD',
                                       #'Punt ReturnsY/R','Punt ReturnsYds',
                                      #'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                      #'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                     #  'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk',# 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', #'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status',
                                             'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status',
                                            'Unnamed: 35_level_0Sk', 'Unnamed: 53_level_0Status'])
            
            print(df_qb2.columns)
            df_qb2 = df_qb2.drop(columns = ['Def InterceptionsInt', 'Def InterceptionsPD', 'Def InterceptionsTD',
               'Def InterceptionsYds',  'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
               'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk','Unnamed: 19_level_0Sk',
               'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
               'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
               'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
               'Unnamed: 25_level_0Status', 'Unnamed: 27_level_0Status',
                'Unnamed: 30_level_0Sk','Unnamed: 30_level_0Status', 'Unnamed: 31_level_0Status',
               'Unnamed: 32_level_0Sk', 'Unnamed: 33_level_0Status',
               'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Status',
               'Unnamed: 39_level_0Sk', 'Unnamed: 39_level_0Status',
               'Unnamed: 40_level_0Sk',
               'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
               'Unnamed: 44_level_0Sk', 'Unnamed: 45_level_0Status',
               'Unnamed: 50_level_0Status',
               'Unnamed: 52_level_0Status', 'Unnamed: 55_level_0Status',
               'Unnamed: 56_level_0Status', 'Unnamed: 57_level_0Status',
               'Unnamed: 58_level_0Status',
               'Unnamed: 62_level_0Status'])
            print(df_qb2.columns)

            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            print(df_qb2.columns)
        elif (position == 'K'):
            print(df_qb2.columns)
            df_qb2 = df_qb2.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                           # 'Def InterceptionsInt', 
                                           #'Def InterceptionsPD', 'Def InterceptionsTD',
                                           #'Def InterceptionsYds', 
                                       #'Kick ReturnsRt','Kick ReturnsTD',
                                       #'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                       #'Punt ReturnsRet', 'Punt ReturnsTD',
                                       #'Punt ReturnsY/R','Punt ReturnsYds',
                                       'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                      'PuntingYds', 
                                        'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                       #    'Unnamed: 21_level_0Status',
                                      # 'Unnamed: 26_level_0Sk', 
                                       #    'Unnamed: 28_level_0Sk',
                                       #'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       #'Unnamed: 32_level_0Status',
                                      # 'Unnamed: 33_level_0Sk', 
                                        #   'Unnamed: 34_level_0Status',
                                      # 'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
                                       #'Unnamed: 36_level_0Sk', #'Unnamed: 37_level_0Sk',
                                       #'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       #'Unnamed: 41_level_0Status',#'Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status',# 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status',# 'Unnamed: 49_level_0Status',
                             #   'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
                                           'Unnamed: 54_level_0Status',
                                            'Unnamed: 11_level_0Sk',# 'Unnamed: 15_level_0Sk',
                                           'Unnamed: 17_level_0Status',# 'Unnamed: 18_level_0Sk',
                                           'Unnamed: 19_level_0Sk',#'Unnamed: 20_level_0Sk',
                                           #'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
                                           'Unnamed: 23_level_0Status', #'Unnamed: 24_level_0Sk',
                                          # 'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
                                          # 'Unnamed: 26_level_0Status', 'Unnamed: 27_level_0Status',
                                          # 'Unnamed: 29_level_0Status', 'Unnamed: 30_level_0Status',
                                           'Unnamed: 31_level_0Status', #'Unnamed: 32_level_0Sk',
                                          # 'Unnamed: 33_level_0Status', 'Unnamed: 35_level_0Sk',
                                           'Unnamed: 36_level_0Status',# 'Unnamed: 37_level_0Sk',
                                           'Unnamed: 37_level_0Status',# 'Unnamed: 39_level_0Status',
                                           'Unnamed: 42_level_0Status',# 'Unnamed: 43_level_0Status',
                                     #      'Unnamed: 45_level_0Status'#,
                                         #   'Unnamed: 50_level_0Status',
                                          # 'Unnamed: 55_level_0Status', 'Unnamed: 58_level_0Status'
                               # 'Unnamed: 61_level_0Status'
                                          ])

            print(df_qb2.columns)
        
            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})

        
        
        df_qb2 = df_qb2[df_qb2['G#'] != '']
        df_qb2 = df_qb2[df_qb2['G#'] != 'G#']
        df_qb2 = df_qb2[~df_qb2['Date'].str.contains("Game")]
        df_qb2 = df_qb2[~df_qb2['GS'].isin(['Did Not Play', 'Inactive',
                                           'Injured Reserve','Non-Football Injury',
                                           'Suspended','Returned from Injured Reserve',
                                           'COVID-19 List','Exempt List',
                                           'Physically Unable to Perform'])]
        
        
        
        
        df_qb2['player_wk_year'] = df_qb2['Player '] + " " + df_qb2['Week'].astype(str) + " " + df_qb2['Year'].astype(str)
        print('yay')
        
        test_df = df[['Player ','Week', 'Year', 'player_wk_year']]
        for i in range(1,36):
            #print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            player_wks_ago = "player " + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "Week"
                curr_yr = "Year"
                test_df[curr_wk] = test_df['Week']
                test_df[curr_yr] = test_df['Year']
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            test_df[player_wks_ago] = test_df['Player '] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)
    
        new_df = pd.DataFrame()
        for i in range(0,36):
            print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df2 = test_df.drop(columns = ['Player ', 'Week', 'Year'])
                new_df = pd.merge(test_df2, df_qb2, how = 'left', on = ['player_wk_year'], suffixes = (None, None))
                print(new_df.head())
            else:
                temp_df = df_qb2.drop(columns = ['Rk','GS','Year','G#'])
                for col in temp_df.columns:
                    new_col = col + " " + str(i) + " wks ago"
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                merge_column = "player " + str(i) + " wks ago"
                temp_df[merge_column] = df_qb2['player_wk_year']
                new_df = pd.merge(new_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
                new_df = new_df.drop_duplicates()
        
        tms = ['Tm','Tm 1 wks ago', 'Tm 2 wks ago',
          'Tm 3 wks ago','Tm 4 wks ago','Tm 5 wks ago','Tm 6 wks ago',
          'Tm 7 wks ago','Tm 8 wks ago','Tm 9 wks ago','Tm 10 wks ago',
          'Tm 11 wks ago','Tm 12 wks ago','Tm 13 wks ago','Tm 14 wks ago',
          'Tm 15 wks ago','Tm 16 wks ago','Tm 17 wks ago','Tm 18 wks ago',
          'Tm 19 wks ago','Tm 20 wks ago','Tm 21 wks ago','Tm 22 wks ago',
          'Tm 23 wks ago','Tm 24 wks ago','Tm 25 wks ago','Tm 26 wks ago',
          'Tm 27 wks ago','Tm 28 wks ago','Tm 29 wks ago','Tm 30 wks ago',
          'Tm 31 wks ago','Tm 32 wks ago','Tm 33 wks ago','Tm 34 wks ago',
          'Tm 35 wks ago']

        opp = ['Opp','Opp 1 wks ago', 'Opp 2 wks ago',
          'Opp 3 wks ago','Opp 4 wks ago','Opp 5 wks ago','Opp 6 wks ago',
          'Opp 7 wks ago','Opp 8 wks ago','Opp 9 wks ago','Opp 10 wks ago',
          'Opp 11 wks ago','Opp 12 wks ago','Opp 13 wks ago','Opp 14 wks ago',
          'Opp 15 wks ago','Opp 16 wks ago','Opp 17 wks ago','Opp 18 wks ago',
          'Opp 19 wks ago','Opp 20 wks ago','Opp 21 wks ago','Opp 22 wks ago',
          'Opp 23 wks ago','Opp 24 wks ago','Opp 25 wks ago','Opp 26 wks ago',
          'Opp 27 wks ago','Opp 28 wks ago','Opp 29 wks ago','Opp 30 wks ago',
          'Opp 31 wks ago','Opp 32 wks ago','Opp 33 wks ago','Opp 34 wks ago',
          'Opp 35 wks ago']
        
        qb_df = new_df
        
        for i in tms:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE',
                               'NOR': 'NO', 'SFO': 'SF', 'SDG':'LAC',
                              'GNB':'GB','TAM':'TB','WAS':'WSH',
                              'KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV', 'IND':'IND', 'SEA':'SEA',
                             'HOU':'HOU', 'ATL':'ATL', 'CIN':'CIN',
                             'PIT':'PIT','NYG':'NYG', 'DET':'DET',
                             'CLE':'CLE','DEN':'DEN','DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI',
                             'BAL':'BAL','JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA',
                             'CAR':'CAR','PHI':'PHI','MIN':'MIN',
                             'TEN':'TEN', 'LAR':'LAR', 'LAC':'LAC'})
        for i in opp:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE', 'NOR': 'NO',
                              'SFO': 'SF', 'SDG':'LAC', 'GNB':'GB','TAM':'TB',
                              'WAS':'WSH','KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV','IND':'IND', 'SEA':'SEA', 'HOU':'HOU',
                             'ATL':'ATL','CIN':'CIN', 'PIT':'PIT', 'NYG':'NYG',
                             'DET':'DET','CLE':'CLE', 'DEN':'DEN', 'DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI','BAL':'BAL',
                             'JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA', 'CAR':'CAR',
                             'PHI':'PHI', 'MIN':'MIN', 'TEN':'TEN', 'LAR':'LAR',
                             'LAC':'LAC'})    
            
        #opp
        opp_wk_year = ['opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']

        wk = ['Week', 'Week 1 wks ago', 'Week 2 wks ago', 'Week 3 wks ago', 'Week 4 wks ago', 'Week 5 wks ago',
              'Week 6 wks ago', 'Week 7 wks ago', 'Week 8 wks ago', 'Week 9 wks ago', 'Week 10 wks ago',
              'Week 11 wks ago', 'Week 12 wks ago', 'Week 13 wks ago', 'Week 14 wks ago', 'Week 15 wks ago',
              'Week 16 wks ago', 'Week 17 wks ago', 'Week 18 wks ago', 'Week 19 wks ago', 'Week 20 wks ago',
              'Week 21 wks ago', 'Week 22 wks ago', 'Week 23 wks ago', 'Week 24 wks ago', 'Week 25 wks ago',
              'Week 26 wks ago', 'Week 27 wks ago', 'Week 28 wks ago', 'Week 29 wks ago', 'Week 30 wks ago',
              'Week 31 wks ago', 'Week 32 wks ago', 'Week 33 wks ago', 'Week 34 wks ago', 'Week 35 wks ago',
              'Week 36 wks ago']

        year = ['Year', 'year 1 wks ago', 'year 2 wks ago', 'year 3 wks ago', 'year 4 wks ago', 'year 5 wks ago',
               'year 6 wks ago', 'year 7 wks ago', 'year 8 wks ago', 'year 9 wks ago', 'year 10 wks ago',
               'year 11 wks ago', 'year 12 wks ago', 'year 13 wks ago', 'year 14 wks ago', 'year 15 wks ago',
               'year 16 wks ago', 'year 17 wks ago', 'year 18 wks ago', 'year 19 wks ago', 'year 20 wks ago',
               'year 21 wks ago', 'year 22 wks ago', 'year 23 wks ago', 'year 24 wks ago', 'year 25 wks ago',
               'year 26 wks ago', 'year 27 wks ago', 'year 28 wks ago', 'year 29 wks ago', 'year 30 wks ago',
               'year 31 wks ago', 'year 32 wks ago', 'year 33 wks ago', 'year 34 wks ago', 'year 35 wks ago',
               'year 36 wks ago']
        
        
        for i in range(0,36):
            print(i)
            #print(opp[i])
            #print(wk[i])
            #print(year[i])
            opp_colname = opp[i]
            wk_colname = wk[i]
            year_colname = year[i]
            opp_wk_year_colname = opp_wk_year[i]
            if (i== 0):
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)
            else:
               # qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str).str[:-2] + " " + qb_df[year_colname].astype(str)
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)

                #if (i == 0):

        test_df = qb_df[['player_wk_year','opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']]

        new_df = pd.DataFrame()

        for i in range(0,36):
            print(i)
            if (i == 0):
                #new_df = df_qb2
                #test_df2 = test_df.drop(columns = [])
                new_df = pd.merge(test_df, def_df, how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
                #print(new_df.head())
            else:
                temp_df = def_df.drop(columns = ['Week','week_num','year','Opp','team_wk_year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'opp' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                temp_df['team_wk_year'] = def_df['team_wk_year']
               # print(temp_df2.head())
                qb_no_null = test_df[[opp_wk_year[i]]]
               # print(qb_no_null.head())
                qb_no_null = qb_no_null[qb_no_null.notnull()]
                qb_no_null = qb_no_null.drop_duplicates()
              #  print(qb_no_null.head())
                qb_no_null = pd.merge(qb_no_null, temp_df, how = 'inner', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None,None))
                #print(qb_no_null.head())
        
                qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                qb_no_null = qb_no_null.drop_duplicates()
                new_df = pd.merge(new_df, qb_no_null, how = 'left', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
                new_df = new_df.drop_duplicates()
        
        new_df = new_df.drop_duplicates()
        
        qb_df2 = qb_df.drop(columns = ['Week', 'Opp', 'opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago',
           'opp 3 wk year ago', 'opp 4 wk year ago', 'opp 5 wk year ago',
           'opp 6 wk year ago', 'opp 7 wk year ago', 'opp 8 wk year ago',
           'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
           'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago',
           'opp 15 wk year ago', 'opp 16 wk year ago', 'opp 17 wk year ago',
           'opp 18 wk year ago', 'opp 19 wk year ago', 'opp 20 wk year ago',
           'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
           'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago',
           'opp 27 wk year ago', 'opp 28 wk year ago', 'opp 29 wk year ago',
           'opp 30 wk year ago', 'opp 31 wk year ago', 'opp 32 wk year ago',
           'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago'])
        

        new_df2 = pd.merge(qb_df2,new_df, how = 'left', on = 'player_wk_year', suffixes = (None,None))
        
        test_df = def_df[['CITY','week_num', 'year', 'team_wk_year']]
        for i in range(1,36):
            #print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            #player_wks_ago = "player " + str(i) + " wks ago"
            team_wk_year = 'team_wk_year ' + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "week_num"
                curr_yr = "year"
                test_df[curr_wk] = test_df['week_num']
                test_df[curr_yr] = test_df['year']
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk] - 1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk]-1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr] -1, test_df[curr_yr])

            test_df[team_wk_year] = test_df['CITY'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

        #test_df.head()
        new_def_df = pd.DataFrame()

        for i in range(0,36):
            #print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df = test_df.drop(columns = ['CITY','week_num','year'])
                new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['team_wk_year'], suffixes = (None, None))
              #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
                print(new_def_df.head())
            else:
                temp_df = def_df.drop(columns = ['Team','CITY','Week','week_num','year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'def matchup' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                merge_column = "team_wk_year " + str(i) + " wks ago"
                temp_df[merge_column] = def_df['team_wk_year']
                #temp_df['team_wk_year'] = def_df['team_wk_year']
               # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
                #new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
        new_def_df2 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
               'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
               'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
               'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
               'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
               'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
               'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
               'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
               'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
               'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
               'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
               'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
               'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
               'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
               'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
               'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
               'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
               'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
               'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
               'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
               'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
               'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
               'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
               'year 35 wks ago', 'Team', #'CITY',
                                                 'Week', 'Sack', 'FR', 'INT', 'DefTD',
               'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD', 'FPts', 'week_num',
               'year', #'Opp',
                                                 'away flag','opp_wk_year'
                                                ])
        
        
        print(df['opp wk year'])
        print(df.shape)
        print(new_df2.shape)
        temp = df[['opp wk year','player_wk_year']]
        new_df2 = new_df2.drop(columns = ['opp wk year'])
        new_df2 = pd.merge(new_df2, temp, how = 'left', on = 'player_wk_year', suffixes = (None,None))
        new_df2 = new_df2.drop(columns = ['CITY', 'Opp'])
       # new_df2['opp wk year'] = df['opp wk year']
        new_df2 = new_df2.drop_duplicates()
        new_df3 = pd.merge(new_df2,new_def_df2,how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None,None))
        
        new_df3 = new_df3.drop(columns = ['def matchup opp_wk_year 10wks ago', 'def matchup opp_wk_year 11wks ago',
             'def matchup opp_wk_year 12wks ago', 'def matchup opp_wk_year 13wks ago',
             'def matchup opp_wk_year 14wks ago', 'def matchup opp_wk_year 15wks ago',
             'def matchup opp_wk_year 16wks ago', 'def matchup opp_wk_year 17wks ago',
             'def matchup opp_wk_year 18wks ago', 'def matchup opp_wk_year 19wks ago',
             'def matchup opp_wk_year 1wks ago', 'def matchup opp_wk_year 20wks ago',
             'def matchup opp_wk_year 21wks ago', 'def matchup opp_wk_year 22wks ago',
             'def matchup opp_wk_year 23wks ago', 'def matchup opp_wk_year 24wks ago',
             'def matchup opp_wk_year 25wks ago', 'def matchup opp_wk_year 26wks ago',
             'def matchup opp_wk_year 27wks ago', 'def matchup opp_wk_year 28wks ago',
             'def matchup opp_wk_year 29wks ago', 'def matchup opp_wk_year 2wks ago',
             'def matchup opp_wk_year 30wks ago', 'def matchup opp_wk_year 31wks ago',
             'def matchup opp_wk_year 32wks ago', 'def matchup opp_wk_year 33wks ago',
             'def matchup opp_wk_year 34wks ago', 'def matchup opp_wk_year 35wks ago',
             'def matchup opp_wk_year 3wks ago', 'def matchup opp_wk_year 4wks ago',
             'def matchup opp_wk_year 5wks ago', 'def matchup opp_wk_year 6wks ago',
             'def matchup opp_wk_year 7wks ago', 'def matchup opp_wk_year 8wks ago',
             'def matchup opp_wk_year 9wks ago', 'opp opp_wk_year 10wks ago',
             'opp opp_wk_year 11wks ago', 'opp opp_wk_year 12wks ago',
             'opp opp_wk_year 13wks ago', 'opp opp_wk_year 14wks ago',
             'opp opp_wk_year 15wks ago', 'opp opp_wk_year 16wks ago',
             'opp opp_wk_year 17wks ago', 'opp opp_wk_year 18wks ago',
             'opp opp_wk_year 19wks ago', 'opp opp_wk_year 1wks ago',
             'opp opp_wk_year 20wks ago', 'opp opp_wk_year 21wks ago',
             'opp opp_wk_year 22wks ago', 'opp opp_wk_year 23wks ago',
             'opp opp_wk_year 24wks ago', 'opp opp_wk_year 25wks ago',
             'opp opp_wk_year 26wks ago', 'opp opp_wk_year 27wks ago',
             'opp opp_wk_year 28wks ago', 'opp opp_wk_year 29wks ago',
             'opp opp_wk_year 2wks ago', 'opp opp_wk_year 30wks ago',
             'opp opp_wk_year 31wks ago', 'opp opp_wk_year 32wks ago',
             'opp opp_wk_year 33wks ago', 'opp opp_wk_year 34wks ago',
             'opp opp_wk_year 35wks ago', 'opp opp_wk_year 3wks ago',
             'opp opp_wk_year 4wks ago', 'opp opp_wk_year 5wks ago',
             'opp opp_wk_year 6wks ago' ,'opp opp_wk_year 7wks ago',
             'opp opp_wk_year 8wks ago', 'opp opp_wk_year 9wks ago', 'opp_wk_year'])
        
        #new_df3['week_match'] = new_df3['CITY'] + " " + new_df3['Opp']
        
       # nfl_sched = nfl_sched[(nfl_sched['Year'] == year) & (nfl_sched['Week'] == week)]
        nfl_sched['Opponent'] = nfl_sched["Opponent"].str.replace("@","")
        nfl_sched['week_match'] = nfl_sched['TEAM'] + " " + nfl_sched['Opponent']
        nfl_sched = nfl_sched[['week_match']]
        
        #new_df4 = pd.merge(new_df3,nfl_sched,how = 'inner', on = ['week_match'], suffixes = (None,None))
        #new_df4 = new_df4.drop(columns = ['week_match'])
        
        
        new_df3.to_csv(save_file)
        
        return new_df3


In [18]:
#now we need to get our current week players and get an oot df
#for the week we care about
def create_curr_wk_oot_qb_v2(nfl_sched,
                          year,
                          week,
                          player_df,
                          position,
                          def_df,
                         save_file):
    
        nfl_sched = nfl_sched[(nfl_sched['Year'] == year) &
                             (nfl_sched['Week'] == week)]
        nfl_sched = nfl_sched.reset_index()
        nfl_sched = nfl_sched.drop(columns = ['index'])
    
        qb = player_df[player_df['Position'] == position]
        qb = qb[qb['Player'] != 'Malik Willis']
        qb = qb[qb['Player'] != 'Brandon Allen']

        qb = qb[qb['Playing_Prd_End'] ==  year]
        qb = qb.reset_index()
        qb = qb.drop(columns = ['index'])
        
        df_qb = pd.DataFrame()
        #for i in range(1000):
        for i in range(len(qb)):
            time.sleep(3.1)
            print(i)
            player = qb['Player'][i]
            print(qb['Player'][i])
            url = qb['Gamelog'][i]
            #print(qb['Player'][i])
            data = requests.get(url).text
            #print(data)
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup)
            dfs = pd.read_html(data)
            df = dfs[0]
            df['Player'] = player
            #print(df.head())    
            #print(df.columns)
            df_qb = pd.concat([df_qb, df], axis=0, ignore_index=True)
        
        df_qb.columns = df_qb.columns.get_level_values(0) + '' +  df_qb.columns.get_level_values(1)
        df_qb = df_qb[['Player','Unnamed: 6_level_0Tm']]
        df_qb = df_qb.rename(columns = {'Unnamed: 6_level_0Tm':'TEAM'})
        df_qb = df_qb.drop_duplicates()
        df_qb['TEAM'] = df_qb['TEAM'].map({'NWE': 'NE',
                               'NOR': 'NO', 'SFO': 'SF', 'SDG':'LAC',
                              'GNB':'GB','TAM':'TB','WAS':'WSH',
                              'KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV', 'IND':'IND', 'SEA':'SEA',
                             'HOU':'HOU', 'ATL':'ATL', 'CIN':'CIN',
                             'PIT':'PIT','NYG':'NYG', 'DET':'DET',
                             'CLE':'CLE','DEN':'DEN','DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI',
                             'BAL':'BAL','JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA',
                             'CAR':'CAR','PHI':'PHI','MIN':'MIN',
                             'TEN':'TEN', 'LAR':'LAR', 'LAC':'LAC'})
        
        df_qb = pd.merge(df_qb, nfl_sched, how = 'inner', on = 'TEAM', 
                        suffixes = (None, None))
        df_qb = df_qb.rename(columns = {'TEAM':'team',
                                       'Player':'Player ',
                                       'Opponent':'opp'})
        df_qb['opp'] = df_qb['opp'].str.replace('@',"")

        df_qb['player_wk_year'] = df_qb['Player '] + " " + df_qb['Week'].astype(str) + " " + df_qb['Year'].astype(str)
        df_qb['opp wk year'] = df_qb['opp'] + " " + df_qb['Week'].astype(str) + " " + df_qb['Year'].astype(str)
        
        df = df_qb[['Week','Year','Player ','team',
                      'opp','player_wk_year','opp wk year']]
        
        
        qb = player_df[player_df['Position'] == position]
        qb = qb[qb['Player'] != 'Malik Willis']
        qb = qb[qb['Playing_Prd_End'] ==  year]
       
        qb = qb.reset_index()
        qb = qb.drop(columns = ['index'])
        
        df_qb2 = pd.DataFrame()
        #for i in range(1000):
        for i in range(len(qb)):
            time.sleep(3.1)

            print(i)
            player = qb['Player'][i]
            print(qb['Player'][i])
            url = qb['Gamelog'][i]
            #print(qb['Player'][i])
            data = requests.get(url).text
            #print(data)
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup)
            dfs = pd.read_html(data)
            df2 = dfs[0]
            df2['Player'] = player
            #print(df.head())    
            #print(df.columns)
            df_qb2 = pd.concat([df_qb2, df2], axis=0, ignore_index=True)
        
        df_qb2.columns = df_qb2.columns.get_level_values(0) + '' +  df_qb2.columns.get_level_values(1)
        if (position == 'QB'):
            print(df_qb2.columns)
            
            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds', 'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                        'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                        'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                        'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                        'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                        'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                        'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})            
            
            df_qb2 = df_qb2[['Player ',
                             'Fumbles FF','Fumbles FL','Fumbles FR','Fumbles Fmb',
                                'Fumbles TD','Fumbles Yds','Off. Snaps Num','Off. Snaps Pct','Passing AY/A',
                                'Passing Att','Passing Cmp','Passing Cmp%','Passing Int','Passing Rate',
                                'Passing Sk','Passing TD','Passing Y/A','Passing Yds',
                                'Rushing Att','Rushing TD','Rushing Y/A','Rushing Yds','ST Snaps Num',
                                'ST Snaps Pct','Scoring 2PM','Scoring Pts','Scoring TD',
                          'Rk', 'GS', 'Year', 'Date', 'G#', 'Week', 'Age', 'Tm',
                          'home_away', 'Opp','Result']]

            print(df_qb2.columns)
        elif (position == 'WR'):
            
        
            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            
            df_qb2 = df_qb2[['Player ',
                            'Fumbles FF','Fumbles FL','Fumbles FR','Fumbles Fmb',
                                    'Fumbles TD','Fumbles Yds','Kick Returns Rt','Kick Returns TD',
                                    'Kick Returns Y/Rt','Kick Returns Yds','Off. Snaps Num','Off. Snaps Pct',
                                    'Punt Returns Ret','Punt Returns TD','Punt Returns Y/R','Punt Returns Yds',
                                    'Receiving Ctch%','Receiving Rec','Receiving TD','Receiving Tgt',
                                    'Receiving Y/R','Receiving Y/Tgt','Receiving Yds','Rushing Att',
                                    'Rushing TD','Rushing Y/A','Rushing Yds','ST Snaps Num',
                                    'ST Snaps Pct','Scoring 2PM','Scoring Pts',#'Scoring Sfty',
                                    'Scoring TD',
                          'Rk', 'GS', 'Year', 'Date', 'G#', 'Week', 'Age', 'Tm',
                          'home_away', 'Opp','Result']]
                
        elif (position == 'TE'):
            
            print(df_qb2.columns)
            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
                
            df_qb2 = df_qb2[['Player ',
                            'Fumbles FF','Fumbles FL','Fumbles FR','Fumbles Fmb',
                                    'Fumbles TD','Fumbles Yds',#'Kick Returns Rt','Kick Returns TD',
                                 #   'Kick Returns Y/Rt','Kick Returns Yds',
                             'Off. Snaps Num','Off. Snaps Pct',
                                   # 'Punt Returns Ret','Punt Returns TD','Punt Returns Y/R','Punt Returns Yds',
                                    'Receiving Ctch%','Receiving Rec','Receiving TD','Receiving Tgt',
                                    'Receiving Y/R','Receiving Y/Tgt','Receiving Yds','Rushing Att',
                                    'Rushing TD','Rushing Y/A','Rushing Yds','ST Snaps Num',
                                    'ST Snaps Pct','Scoring 2PM','Scoring Pts',#'Scoring Sfty',
                                    'Scoring TD',
                          'Rk', 'GS', 'Year', 'Date', 'G#', 'Week', 'Age', 'Tm',
                          'home_away', 'Opp','Result']]
            
        elif (position == 'RB'):
            print(df_qb2.columns)
 
            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
    
            df_qb2 = df_qb2[['Player ',
                            'Fumbles FF','Fumbles FL','Fumbles FR','Fumbles Fmb',
                                    'Fumbles TD','Fumbles Yds','Kick Returns Rt','Kick Returns TD',
                                    'Kick Returns Y/Rt','Kick Returns Yds','Off. Snaps Num','Off. Snaps Pct',
                                    'Punt Returns Ret','Punt Returns TD','Punt Returns Y/R','Punt Returns Yds',
                                    'Receiving Ctch%','Receiving Rec','Receiving TD','Receiving Tgt',
                                    'Receiving Y/R','Receiving Y/Tgt','Receiving Yds','Rushing Att',
                                    'Rushing TD','Rushing Y/A','Rushing Yds','ST Snaps Num',
                                    'ST Snaps Pct','Scoring 2PM','Scoring Pts',#'Scoring Sfty',
                                    'Scoring TD',
                          'Rk', 'GS', 'Year', 'Date', 'G#', 'Week', 'Age', 'Tm',
                          'home_away', 'Opp','Result']]    
            print(df_qb2.columns)
        elif (position == 'K'):
            print(df_qb2.columns)

            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                      'FumblesFL':'Fumbles FL',
                      'FumblesFR':'Fumbles FR',
                      'FumblesFmb':'Fumbles Fmb',
                      'FumblesTD':'Fumbles TD',
                            'FumblesYds':'Fumbles Yds',
                                   'Kick ReturnsRt':'Kick Returns Rt',
                                'Kick ReturnsTD':'Kick Returns TD',
                           'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                'Kick ReturnsYds':'Kick Returns Yds',
                                'Off. SnapsNum':'Off. Snaps Num',
                            'Off. SnapsPct':'Off. Snaps Pct',
                            'Player':'Player ', 
                                'Punt ReturnsRet':'Punt Returns Ret',
                                'Punt ReturnsTD':'Punt Returns TD',
                           'Punt ReturnsY/R':'Punt Returns Y/R',
                                'Punt ReturnsYds':'Punt Returns Yds',
                                'ReceivingCtch%':'Receiving Ctch%',
                                'ReceivingRec':'Receiving Rec',
                                'ReceivingTD':'Receiving TD',
                           'ReceivingTgt':'Receiving Tgt',
                                'ReceivingY/R':'Receiving Y/R',
                                'ReceivingY/Tgt':'Receiving Y/Tgt',
                                'ReceivingYds':'Receiving Yds',
                                'RushingAtt':'Rushing Att',
                            'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                            'RushingYds':'Rushing Yds',
                            'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                            'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                            'ScoringTD':'Scoring TD',
                            'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                            'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                            'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                            'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                            'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                      'Unnamed: 8_level_0Opp':'Opp',
                            'Unnamed: 9_level_0Result':'Result'})
            
            df_qb2 = df_qb2[['Player ','ST Snaps Num', 'ST Snaps Pct', 'Scoring Pts',
                           'ScoringXPM','ScoringXPA','ScoringXP%','ScoringFGM','ScoringFGA',
                           'ScoringFG%',#'ScoringTD',
                          'Rk', 'GS', 'Year', 'Date', 'G#', 'Week', 'Age', 'Tm',
                          'home_away', 'Opp','Result']]


        
        df_qb2 = df_qb2[df_qb2['G#'] != '']
        df_qb2 = df_qb2[df_qb2['G#'] != 'G#']
        df_qb2 = df_qb2[~df_qb2['Date'].str.contains("Game")]
        df_qb2 = df_qb2[~df_qb2['GS'].isin(['Did Not Play', 'Inactive',
                                           'Injured Reserve','Non-Football Injury',
                                           'Suspended','Returned from Injured Reserve',
                                           'COVID-19 List','Exempt List',
                                           'Physically Unable to Perform'])]
        
        
        
        
        df_qb2['player_wk_year'] = df_qb2['Player '] + " " + df_qb2['Week'].astype(str) + " " + df_qb2['Year'].astype(str)
        print('yay')
        
        test_df = df[['Player ','Week', 'Year', 'player_wk_year']]
        for i in range(1,36):
            #print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            player_wks_ago = "player " + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "Week"
                curr_yr = "Year"
                test_df[curr_wk] = test_df['Week']
                test_df[curr_yr] = test_df['Year']
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            test_df[player_wks_ago] = test_df['Player '] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)
    
        new_df = pd.DataFrame()
        for i in range(0,36):
            print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df2 = test_df.drop(columns = ['Player ', 'Week', 'Year'])
                new_df = pd.merge(test_df2, df_qb2, how = 'left', on = ['player_wk_year'], suffixes = (None, None))
                print(new_df.head())
            else:
                temp_df = df_qb2.drop(columns = ['Rk','GS','Year','G#'])
                for col in temp_df.columns:
                    new_col = col + " " + str(i) + " wks ago"
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                merge_column = "player " + str(i) + " wks ago"
                temp_df[merge_column] = df_qb2['player_wk_year']
                new_df = pd.merge(new_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
                new_df = new_df.drop_duplicates()
        
        tms = ['Tm','Tm 1 wks ago', 'Tm 2 wks ago',
          'Tm 3 wks ago','Tm 4 wks ago','Tm 5 wks ago','Tm 6 wks ago',
          'Tm 7 wks ago','Tm 8 wks ago','Tm 9 wks ago','Tm 10 wks ago',
          'Tm 11 wks ago','Tm 12 wks ago','Tm 13 wks ago','Tm 14 wks ago',
          'Tm 15 wks ago','Tm 16 wks ago','Tm 17 wks ago','Tm 18 wks ago',
          'Tm 19 wks ago','Tm 20 wks ago','Tm 21 wks ago','Tm 22 wks ago',
          'Tm 23 wks ago','Tm 24 wks ago','Tm 25 wks ago','Tm 26 wks ago',
          'Tm 27 wks ago','Tm 28 wks ago','Tm 29 wks ago','Tm 30 wks ago',
          'Tm 31 wks ago','Tm 32 wks ago','Tm 33 wks ago','Tm 34 wks ago',
          'Tm 35 wks ago']

        opp = ['Opp','Opp 1 wks ago', 'Opp 2 wks ago',
          'Opp 3 wks ago','Opp 4 wks ago','Opp 5 wks ago','Opp 6 wks ago',
          'Opp 7 wks ago','Opp 8 wks ago','Opp 9 wks ago','Opp 10 wks ago',
          'Opp 11 wks ago','Opp 12 wks ago','Opp 13 wks ago','Opp 14 wks ago',
          'Opp 15 wks ago','Opp 16 wks ago','Opp 17 wks ago','Opp 18 wks ago',
          'Opp 19 wks ago','Opp 20 wks ago','Opp 21 wks ago','Opp 22 wks ago',
          'Opp 23 wks ago','Opp 24 wks ago','Opp 25 wks ago','Opp 26 wks ago',
          'Opp 27 wks ago','Opp 28 wks ago','Opp 29 wks ago','Opp 30 wks ago',
          'Opp 31 wks ago','Opp 32 wks ago','Opp 33 wks ago','Opp 34 wks ago',
          'Opp 35 wks ago']
        
        qb_df = new_df
        
        for i in tms:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE',
                               'NOR': 'NO', 'SFO': 'SF', 'SDG':'LAC',
                              'GNB':'GB','TAM':'TB','WAS':'WSH',
                              'KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV', 'IND':'IND', 'SEA':'SEA',
                             'HOU':'HOU', 'ATL':'ATL', 'CIN':'CIN',
                             'PIT':'PIT','NYG':'NYG', 'DET':'DET',
                             'CLE':'CLE','DEN':'DEN','DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI',
                             'BAL':'BAL','JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA',
                             'CAR':'CAR','PHI':'PHI','MIN':'MIN',
                             'TEN':'TEN', 'LAR':'LAR', 'LAC':'LAC'})
        for i in opp:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE', 'NOR': 'NO',
                              'SFO': 'SF', 'SDG':'LAC', 'GNB':'GB','TAM':'TB',
                              'WAS':'WSH','KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV','IND':'IND', 'SEA':'SEA', 'HOU':'HOU',
                             'ATL':'ATL','CIN':'CIN', 'PIT':'PIT', 'NYG':'NYG',
                             'DET':'DET','CLE':'CLE', 'DEN':'DEN', 'DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI','BAL':'BAL',
                             'JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA', 'CAR':'CAR',
                             'PHI':'PHI', 'MIN':'MIN', 'TEN':'TEN', 'LAR':'LAR',
                             'LAC':'LAC'})    
            
        #opp
        opp_wk_year = ['opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']

        wk = ['Week', 'Week 1 wks ago', 'Week 2 wks ago', 'Week 3 wks ago', 'Week 4 wks ago', 'Week 5 wks ago',
              'Week 6 wks ago', 'Week 7 wks ago', 'Week 8 wks ago', 'Week 9 wks ago', 'Week 10 wks ago',
              'Week 11 wks ago', 'Week 12 wks ago', 'Week 13 wks ago', 'Week 14 wks ago', 'Week 15 wks ago',
              'Week 16 wks ago', 'Week 17 wks ago', 'Week 18 wks ago', 'Week 19 wks ago', 'Week 20 wks ago',
              'Week 21 wks ago', 'Week 22 wks ago', 'Week 23 wks ago', 'Week 24 wks ago', 'Week 25 wks ago',
              'Week 26 wks ago', 'Week 27 wks ago', 'Week 28 wks ago', 'Week 29 wks ago', 'Week 30 wks ago',
              'Week 31 wks ago', 'Week 32 wks ago', 'Week 33 wks ago', 'Week 34 wks ago', 'Week 35 wks ago',
              'Week 36 wks ago']

        year = ['Year', 'year 1 wks ago', 'year 2 wks ago', 'year 3 wks ago', 'year 4 wks ago', 'year 5 wks ago',
               'year 6 wks ago', 'year 7 wks ago', 'year 8 wks ago', 'year 9 wks ago', 'year 10 wks ago',
               'year 11 wks ago', 'year 12 wks ago', 'year 13 wks ago', 'year 14 wks ago', 'year 15 wks ago',
               'year 16 wks ago', 'year 17 wks ago', 'year 18 wks ago', 'year 19 wks ago', 'year 20 wks ago',
               'year 21 wks ago', 'year 22 wks ago', 'year 23 wks ago', 'year 24 wks ago', 'year 25 wks ago',
               'year 26 wks ago', 'year 27 wks ago', 'year 28 wks ago', 'year 29 wks ago', 'year 30 wks ago',
               'year 31 wks ago', 'year 32 wks ago', 'year 33 wks ago', 'year 34 wks ago', 'year 35 wks ago',
               'year 36 wks ago']
        
        
        for i in range(0,36):
            print(i)
            #print(opp[i])
            #print(wk[i])
            #print(year[i])
            opp_colname = opp[i]
            wk_colname = wk[i]
            year_colname = year[i]
            opp_wk_year_colname = opp_wk_year[i]
            if (i== 0):
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)
            else:
               # qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str).str[:-2] + " " + qb_df[year_colname].astype(str)
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)

                #if (i == 0):

        test_df = qb_df[['player_wk_year','opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']]

        new_df = pd.DataFrame()

        for i in range(0,36):
            print(i)
            if (i == 0):
                #new_df = df_qb2
                #test_df2 = test_df.drop(columns = [])
                new_df = pd.merge(test_df, def_df, how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
                #print(new_df.head())
            else:
                temp_df = def_df.drop(columns = ['Week','week_num','year','Opp','team_wk_year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'opp' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                temp_df['team_wk_year'] = def_df['team_wk_year']
               # print(temp_df2.head())
                qb_no_null = test_df[[opp_wk_year[i]]]
               # print(qb_no_null.head())
                qb_no_null = qb_no_null[qb_no_null.notnull()]
                qb_no_null = qb_no_null.drop_duplicates()
              #  print(qb_no_null.head())
                qb_no_null = pd.merge(qb_no_null, temp_df, how = 'inner', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None,None))
                #print(qb_no_null.head())
        
                qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                qb_no_null = qb_no_null.drop_duplicates()
                new_df = pd.merge(new_df, qb_no_null, how = 'left', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
                new_df = new_df.drop_duplicates()
        
        new_df = new_df.drop_duplicates()
        
        qb_df2 = qb_df.drop(columns = ['Week', 'Opp', 'opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago',
           'opp 3 wk year ago', 'opp 4 wk year ago', 'opp 5 wk year ago',
           'opp 6 wk year ago', 'opp 7 wk year ago', 'opp 8 wk year ago',
           'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
           'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago',
           'opp 15 wk year ago', 'opp 16 wk year ago', 'opp 17 wk year ago',
           'opp 18 wk year ago', 'opp 19 wk year ago', 'opp 20 wk year ago',
           'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
           'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago',
           'opp 27 wk year ago', 'opp 28 wk year ago', 'opp 29 wk year ago',
           'opp 30 wk year ago', 'opp 31 wk year ago', 'opp 32 wk year ago',
           'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago'])
        

        new_df2 = pd.merge(qb_df2,new_df, how = 'left', on = 'player_wk_year', suffixes = (None,None))
        
        test_df = def_df[['CITY','week_num', 'year', 'team_wk_year']]
        for i in range(1,36):
            #print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            #player_wks_ago = "player " + str(i) + " wks ago"
            team_wk_year = 'team_wk_year ' + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "week_num"
                curr_yr = "year"
                test_df[curr_wk] = test_df['week_num']
                test_df[curr_yr] = test_df['year']
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk] - 1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk]-1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr] -1, test_df[curr_yr])

            test_df[team_wk_year] = test_df['CITY'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

        #test_df.head()
        new_def_df = pd.DataFrame()

        for i in range(0,36):
            #print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df = test_df.drop(columns = ['CITY','week_num','year'])
                new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['team_wk_year'], suffixes = (None, None))
              #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
                print(new_def_df.head())
            else:
                temp_df = def_df.drop(columns = ['Team','CITY','Week','week_num','year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'def matchup' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                merge_column = "team_wk_year " + str(i) + " wks ago"
                temp_df[merge_column] = def_df['team_wk_year']
                #temp_df['team_wk_year'] = def_df['team_wk_year']
               # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
                #new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
        new_def_df2 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
               'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
               'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
               'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
               'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
               'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
               'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
               'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
               'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
               'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
               'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
               'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
               'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
               'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
               'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
               'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
               'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
               'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
               'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
               'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
               'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
               'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
               'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
               'year 35 wks ago', 'Team', #'CITY',
                                                 'Week', 'Sack', 'FR', 'INT', 'DefTD',
               'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD', 'FPts', 'week_num',
               'year', #'Opp',
                                                 'away flag','opp_wk_year'
                                                ])
        
        
        print(df['opp wk year'])
        print(df.shape)
        print(new_df2.shape)
        temp = df[['opp wk year','player_wk_year']]
        new_df2 = new_df2.drop(columns = ['opp wk year'])
        new_df2 = pd.merge(new_df2, temp, how = 'left', on = 'player_wk_year', suffixes = (None,None))
        new_df2 = new_df2.drop(columns = ['CITY', 'Opp'])
       # new_df2['opp wk year'] = df['opp wk year']
        new_df2 = new_df2.drop_duplicates()
        new_df3 = pd.merge(new_df2,new_def_df2,how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None,None))
        
        new_df3 = new_df3.drop(columns = ['def matchup opp_wk_year 10wks ago', 'def matchup opp_wk_year 11wks ago',
             'def matchup opp_wk_year 12wks ago', 'def matchup opp_wk_year 13wks ago',
             'def matchup opp_wk_year 14wks ago', 'def matchup opp_wk_year 15wks ago',
             'def matchup opp_wk_year 16wks ago', 'def matchup opp_wk_year 17wks ago',
             'def matchup opp_wk_year 18wks ago', 'def matchup opp_wk_year 19wks ago',
             'def matchup opp_wk_year 1wks ago', 'def matchup opp_wk_year 20wks ago',
             'def matchup opp_wk_year 21wks ago', 'def matchup opp_wk_year 22wks ago',
             'def matchup opp_wk_year 23wks ago', 'def matchup opp_wk_year 24wks ago',
             'def matchup opp_wk_year 25wks ago', 'def matchup opp_wk_year 26wks ago',
             'def matchup opp_wk_year 27wks ago', 'def matchup opp_wk_year 28wks ago',
             'def matchup opp_wk_year 29wks ago', 'def matchup opp_wk_year 2wks ago',
             'def matchup opp_wk_year 30wks ago', 'def matchup opp_wk_year 31wks ago',
             'def matchup opp_wk_year 32wks ago', 'def matchup opp_wk_year 33wks ago',
             'def matchup opp_wk_year 34wks ago', 'def matchup opp_wk_year 35wks ago',
             'def matchup opp_wk_year 3wks ago', 'def matchup opp_wk_year 4wks ago',
             'def matchup opp_wk_year 5wks ago', 'def matchup opp_wk_year 6wks ago',
             'def matchup opp_wk_year 7wks ago', 'def matchup opp_wk_year 8wks ago',
             'def matchup opp_wk_year 9wks ago', 'opp opp_wk_year 10wks ago',
             'opp opp_wk_year 11wks ago', 'opp opp_wk_year 12wks ago',
             'opp opp_wk_year 13wks ago', 'opp opp_wk_year 14wks ago',
             'opp opp_wk_year 15wks ago', 'opp opp_wk_year 16wks ago',
             'opp opp_wk_year 17wks ago', 'opp opp_wk_year 18wks ago',
             'opp opp_wk_year 19wks ago', 'opp opp_wk_year 1wks ago',
             'opp opp_wk_year 20wks ago', 'opp opp_wk_year 21wks ago',
             'opp opp_wk_year 22wks ago', 'opp opp_wk_year 23wks ago',
             'opp opp_wk_year 24wks ago', 'opp opp_wk_year 25wks ago',
             'opp opp_wk_year 26wks ago', 'opp opp_wk_year 27wks ago',
             'opp opp_wk_year 28wks ago', 'opp opp_wk_year 29wks ago',
             'opp opp_wk_year 2wks ago', 'opp opp_wk_year 30wks ago',
             'opp opp_wk_year 31wks ago', 'opp opp_wk_year 32wks ago',
             'opp opp_wk_year 33wks ago', 'opp opp_wk_year 34wks ago',
             'opp opp_wk_year 35wks ago', 'opp opp_wk_year 3wks ago',
             'opp opp_wk_year 4wks ago', 'opp opp_wk_year 5wks ago',
             'opp opp_wk_year 6wks ago' ,'opp opp_wk_year 7wks ago',
             'opp opp_wk_year 8wks ago', 'opp opp_wk_year 9wks ago', 'opp_wk_year'])
        
        #new_df3['week_match'] = new_df3['CITY'] + " " + new_df3['Opp']
        
       # nfl_sched = nfl_sched[(nfl_sched['Year'] == year) & (nfl_sched['Week'] == week)]
        nfl_sched['Opponent'] = nfl_sched["Opponent"].str.replace("@","")
        nfl_sched['week_match'] = nfl_sched['TEAM'] + " " + nfl_sched['Opponent']
        nfl_sched = nfl_sched[['week_match']]
        
        #new_df4 = pd.merge(new_df3,nfl_sched,how = 'inner', on = ['week_match'], suffixes = (None,None))
        #new_df4 = new_df4.drop(columns = ['week_match'])
        
        
        new_df3.to_csv(save_file)
        
        return new_df3


In [7]:
def def_feat_eng_df_creation(def_df,
                                 file_save):
                
    def_df['Opp'] = def_df["Opp"].str.replace("@","")
    def_df['opp_wk_year'] = def_df['Opp'] + " " + def_df['week_num'].astype(str) + " " + def_df['year'].astype(str)    

    tms = ['Tm','Tm 1 wks ago', 'Tm 2 wks ago',
              'Tm 3 wks ago','Tm 4 wks ago','Tm 5 wks ago','Tm 6 wks ago',
              'Tm 7 wks ago','Tm 8 wks ago','Tm 9 wks ago','Tm 10 wks ago',
              'Tm 11 wks ago','Tm 12 wks ago','Tm 13 wks ago','Tm 14 wks ago',
              'Tm 15 wks ago','Tm 16 wks ago','Tm 17 wks ago','Tm 18 wks ago',
              'Tm 19 wks ago','Tm 20 wks ago','Tm 21 wks ago','Tm 22 wks ago',
              'Tm 23 wks ago','Tm 24 wks ago','Tm 25 wks ago','Tm 26 wks ago',
              'Tm 27 wks ago','Tm 28 wks ago','Tm 29 wks ago','Tm 30 wks ago',
              'Tm 31 wks ago','Tm 32 wks ago','Tm 33 wks ago','Tm 34 wks ago',
              'Tm 35 wks ago']

    opp = ['Opp','Opp 1 wks ago', 'Opp 2 wks ago',
              'Opp 3 wks ago','Opp 4 wks ago','Opp 5 wks ago','Opp 6 wks ago',
              'Opp 7 wks ago','Opp 8 wks ago','Opp 9 wks ago','Opp 10 wks ago',
              'Opp 11 wks ago','Opp 12 wks ago','Opp 13 wks ago','Opp 14 wks ago',
              'Opp 15 wks ago','Opp 16 wks ago','Opp 17 wks ago','Opp 18 wks ago',
              'Opp 19 wks ago','Opp 20 wks ago','Opp 21 wks ago','Opp 22 wks ago',
              'Opp 23 wks ago','Opp 24 wks ago','Opp 25 wks ago','Opp 26 wks ago',
              'Opp 27 wks ago','Opp 28 wks ago','Opp 29 wks ago','Opp 30 wks ago',
              'Opp 31 wks ago','Opp 32 wks ago','Opp 33 wks ago','Opp 34 wks ago',
              'Opp 35 wks ago']

        #opp
    opp_wk_year = ['opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']

    wk = ['Week', 'Week 1 wks ago', 'Week 2 wks ago', 'Week 3 wks ago', 'Week 4 wks ago', 'Week 5 wks ago',
              'Week 6 wks ago', 'Week 7 wks ago', 'Week 8 wks ago', 'Week 9 wks ago', 'Week 10 wks ago',
              'Week 11 wks ago', 'Week 12 wks ago', 'Week 13 wks ago', 'Week 14 wks ago', 'Week 15 wks ago',
              'Week 16 wks ago', 'Week 17 wks ago', 'Week 18 wks ago', 'Week 19 wks ago', 'Week 20 wks ago',
              'Week 21 wks ago', 'Week 22 wks ago', 'Week 23 wks ago', 'Week 24 wks ago', 'Week 25 wks ago',
              'Week 26 wks ago', 'Week 27 wks ago', 'Week 28 wks ago', 'Week 29 wks ago', 'Week 30 wks ago',
              'Week 31 wks ago', 'Week 32 wks ago', 'Week 33 wks ago', 'Week 34 wks ago', 'Week 35 wks ago',
              'Week 36 wks ago']

    year = ['Year', 'year 1 wks ago', 'year 2 wks ago', 'year 3 wks ago', 'year 4 wks ago', 'year 5 wks ago',
               'year 6 wks ago', 'year 7 wks ago', 'year 8 wks ago', 'year 9 wks ago', 'year 10 wks ago',
               'year 11 wks ago', 'year 12 wks ago', 'year 13 wks ago', 'year 14 wks ago', 'year 15 wks ago',
               'year 16 wks ago', 'year 17 wks ago', 'year 18 wks ago', 'year 19 wks ago', 'year 20 wks ago',
               'year 21 wks ago', 'year 22 wks ago', 'year 23 wks ago', 'year 24 wks ago', 'year 25 wks ago',
               'year 26 wks ago', 'year 27 wks ago', 'year 28 wks ago', 'year 29 wks ago', 'year 30 wks ago',
               'year 31 wks ago', 'year 32 wks ago', 'year 33 wks ago', 'year 34 wks ago', 'year 35 wks ago',
               'year 36 wks ago']
        
    test_df = def_df[['CITY','week_num', 'year', 'team_wk_year']]
    for i in range(1,36):
        print(i)
        wks_ago = i
        week_num_wks_ago = "week num " + str(i) + " wks ago"
        year_wks_ago = "year " + str(i) + " wks ago"
        player_wks_ago = "team_wk_year " + str(i) + " wks ago"
        if(i == 1):
            curr_wk = "week_num"
            curr_yr = "year"
            test_df[curr_wk] = test_df['week_num']
            test_df[curr_yr] = test_df['year']
            test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
            test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
        else:
            curr_wk = "week num " + str(i-1) + " wks ago"
            curr_yr = "year " + str(i-1) + " wks ago"
            test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
            test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
        test_df[player_wks_ago] = test_df['CITY'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

            
    new_def_df = pd.DataFrame()

    for i in range(0,36):
        print(i)
        if (i == 0):
            #new_df = df_qb2
            test_df = test_df.drop(columns = ['CITY','week_num','year'])
            new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['team_wk_year'], suffixes = (None, None))
          #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
            print(new_def_df.head())
        else:
            temp_df = def_df.drop(columns = ['Team','CITY','Week','week_num','year'])
      #  temp_df2 = temp_df
            for col in temp_df.columns:
                new_col = 'def matchup' + " " + col + " " + str(i) + 'wks ago'
                temp_df[new_col] = temp_df[col]
                temp_df = temp_df.drop(columns = [col])
        #merge_column = "player " + str(i) + " wks ago"
        #merge_column = 
            merge_column = "team_wk_year " + str(i) + " wks ago"
            temp_df[merge_column] = def_df['team_wk_year']
        #temp_df['team_wk_year'] = def_df['team_wk_year']
       # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
            new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
        #new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
    new_def_df2 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
           'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
           'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
           'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
           'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
           'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
           'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
           'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
           'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
           'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
           'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
           'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
           'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
           'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
           'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
           'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
           'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
           'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
           'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
           'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
           'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
           'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
           'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
           'year 35 wks ago', 'Team', 'CITY', 
                                #  'week_num',
                                   'Sack', 'FR', 'INT', 'DefTD',
           'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD',
                                         #'FPts', 
                                         'PA',
        #                                 'week_num',
       #'year'
                               #   , 'Opp',
                                         'away flag'
                                 ]) 
            
    test_df = def_df[['Opp','week_num', 'year', 'opp_wk_year']]
    for i in range(1,36):
        print(i)
        wks_ago = i
        week_num_wks_ago = "week num " + str(i) + " wks ago"
        year_wks_ago = "year " + str(i) + " wks ago"
        #player_wks_ago = "player " + str(i) + " wks ago"
        team_wk_year = 'opp_wk_year ' + str(i) + " wks ago"
        if(i == 1):
            curr_wk = "week_num"
            curr_yr = "year"
            test_df[curr_wk] = test_df['week_num']
            test_df[curr_yr] = test_df['year']
            test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
            test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
        else:
            curr_wk = "week num " + str(i-1) + " wks ago"
            curr_yr = "year " + str(i-1) + " wks ago"
            test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
            test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])

        test_df[team_wk_year] = test_df['Opp'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

    #print(week_num_wks_ago)
#    for j in range(len(df_qb2)):
 #       if(i == 1):
 #           test_df[curr_wk] = test_df['Week']
 #           test_df[curr_yr] = test_df['Year']
 #           test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk] - 1)
 #           test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
 #       else:
 #           test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk]-1)
 #           test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr] -1, test_df[curr_yr])
            
 #       test_df[player_wks_ago] = test_df['Player '] + " " + str(test_df[week_num_wks_ago]) + " " + str(test_df[year_wks_ago])
            
            
    test_df.head()
    test_df = test_df.drop_duplicates()    
        
    new_def_df = pd.DataFrame()
    print("test")
    for i in range(0,36):
        print(i)
        if (i == 0):
        #new_df = df_qb2
            test_df = test_df.drop(columns = ['Opp','week_num','year'])
            new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['opp_wk_year'], suffixes = (None, None))
      #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
            print(new_def_df.head())
        else:
            temp_df = def_df.drop(columns = ['Team','Opp','Week','week_num','year'])
      #  temp_df2 = temp_df
            for col in temp_df.columns:
                new_col = 'opp matchup' + " " + col + " " + str(i) + 'wks ago'
                temp_df[new_col] = temp_df[col]
                temp_df = temp_df.drop(columns = [col])
        #merge_column = "player " + str(i) + " wks ago"
        #merge_column = 
            merge_column = "opp_wk_year " + str(i) + " wks ago"
            temp_df[merge_column] = def_df['opp_wk_year']
        #temp_df['team_wk_year'] = def_df['team_wk_year']
       # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
            new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
            new_def_df = new_def_df.drop_duplicates(subset=merge_column, keep = 'first')
            print(new_def_df.shape)
        #new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
    new_def_df3 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
       'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
       'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
       'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
       'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
       'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
       'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
       'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
       'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
       'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
       'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
       'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
       'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
       'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
       'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
       'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
       'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
       'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
       'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
       'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
       'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
       'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
       'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
       'year 35 wks ago', 'Team', 'CITY', 'Week', 'Sack', 'FR', 'INT', 'DefTD',
       'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD',
                                         'FPts', 'team_wk_year',
                                         'PA',
                                         'week_num',
       'year', 'Opp',
                                         'away flag'])
       
        
    new_def_df4 = pd.merge(new_def_df2, new_def_df3, how = 'inner', on = 'opp_wk_year', suffixes = (None,None))

   
    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = new_def_df4[['team_wk_year']]    
    temp_df2 = pd.DataFrame()     
    features = ['opp matchup Sack', 'opp matchup FR', 'opp matchup INT', 'opp matchup DefTD', 'opp matchup PA',
            'opp matchup PaYD', 'opp matchup RuYd',
       'opp matchup Safety', 'opp matchup KickTD', 'opp matchup FPts', 'opp matchup away flag',
           'opp matchup PA']

    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + "wks ago"
            new_def_df4[fillna_col] = new_def_df4[fillna_col].fillna(0)
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = new_def_df4[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     

        
    new_def_df4 = pd.merge(new_def_df4, temp_df, how = 'inner', on= 'team_wk_year', suffixes = (None,None))        
    
    
    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = new_def_df4[['team_wk_year']]    
    temp_df2 = pd.DataFrame()     
    features = ['def matchup Sack',
    'def matchup FR',
    'def matchup INT',
    'def matchup DefTD',
    'def matchup PA',
    'def matchup PaYD',
    'def matchup RuYd',
    'def matchup Safety',
    'def matchup KickTD',
    'def matchup FPts',
    'def matchup Opp',
    'def matchup away flag',
           'def matchup PA']

    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + "wks ago"
            new_def_df4[fillna_col] = new_def_df4[fillna_col].fillna(0)
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = new_def_df4[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     


    new_def_df4 = pd.merge(new_def_df4, temp_df, how = 'inner', on= 'team_wk_year', suffixes = (None,None))
    new_def_df4 = new_def_df4.drop_duplicates()
    new_def_df4.to_csv(file_save)
    
    return new_def_df4

In [8]:
def def_curr_wk_oot(def_df,
                    nfl_sched,
                    curr_wk,
                    curr_yr,
                    file_save):
    nfl_sched = nfl_sched[(nfl_sched['Year'] == curr_yr) & 
                         (nfl_sched['Week'] == curr_wk)]
    
    nfl_sched = nfl_sched.rename(columns = {'Year':'year',
                                           'TEAM':'CITY',
                                           'Opponent':'Opp'})
    nfl_sched['week_num'] = nfl_sched['Week']
    nfl_sched['Opp'] = nfl_sched["Opp"].str.replace("@","")
    nfl_sched['opp wk year'] = nfl_sched['Opp'] + " " + nfl_sched['week_num'].astype(str) + " " + nfl_sched['year'].astype(str)    
   

    nfl_sched['Team'] = nfl_sched['CITY'].map({'ARI':'Cardinals','SF':'49ers','NO':'Saints',
                                   'CAR':'Panthers','DAL':'Cowboys','HOU':'Texans',
                                   'LV':'Raiders','LAR':'Rams','DET':'Lions',
                                   'KC':'Chiefs','WSH':'Commanders','BAL':'Ravens',
                                   'IND':'Colts','PIT':'Steelers','CIN':'Bengals',
                                   'SEA':'Seahawks','MIA':'Dolphins','MIN':'Vikings',
                                   'NE':'Patriots','NYG':'Giants','TEN':'Titans',
                                   'DEN':'Broncos', 'NYJ':'Jets','PHI':'Eagles',
                                   'LAC':'Chargers','TB':'Buccaneers','BUF':'Bills',
                                   'CLE':'Browns','ATL':'Falcons','JAX':'Jaguars',
                                   'CHI':'Bears','GB':'Packers'})
    
    nfl_sched['Sack'] = None
    nfl_sched['FR'] = None
    nfl_sched['INT'] = None
    nfl_sched['DefTD'] = None
    nfl_sched['PA'] = None
    nfl_sched['PaYD'] = None
    nfl_sched['RuYd'] = None
    nfl_sched['Safety'] = None
    nfl_sched['KickTD'] = None
    nfl_sched['FPts'] = None
    nfl_sched['away flag'] = None

    def_df['Opp'] = def_df['Opp'].str.replace('@','')
    def_df['opp wk year'] = def_df['Opp'] + " " + def_df['week_num'].astype(str) + " " + def_df['year'].astype(str)
    #def_df = def_df.append(nfl_sched)
    
    def_df = def_df.append(nfl_sched)
    def_df = def_df.drop(columns = ['opp wk year'])
    
    def_df_wk = def_feat_eng_df_creation(def_df = def_df,
                                 file_save = file_save)
    
    filt_wk = str(curr_wk) + " " + str(curr_yr)
    
    def_df_wk = def_df_wk[def_df_wk['team_wk_year'].str.contains(filt_wk)]
    
    #def_df_wk.to_cvs(file_save)
    
    return def_df_wk

In [9]:
def def_curr_wk_for_positions(def_df,
                    nfl_sched,
                    curr_wk,
                    curr_yr):
    nfl_sched = nfl_sched[(nfl_sched['Year'] == curr_yr) & 
                         (nfl_sched['Week'] == curr_wk)]
    
    nfl_sched = nfl_sched.rename(columns = {'Year':'year',
                                           'TEAM':'CITY',
                                           'Opponent':'Opp'})
    nfl_sched['week_num'] = nfl_sched['Week']
    nfl_sched['Opp'] = nfl_sched["Opp"].str.replace("@","")
    nfl_sched['opp_wk_year'] = nfl_sched['Opp'] + " " + nfl_sched['week_num'].astype(str) + " " + nfl_sched['year'].astype(str)    
   

    nfl_sched['Team'] = nfl_sched['CITY'].map({'ARI':'Cardinals','SF':'49ers','NO':'Saints',
                                   'CAR':'Panthers','DAL':'Cowboys','HOU':'Texans',
                                   'LV':'Raiders','LAR':'Rams','DET':'Lions',
                                   'KC':'Chiefs','WSH':'Commanders','BAL':'Ravens',
                                   'IND':'Colts','PIT':'Steelers','CIN':'Bengals',
                                   'SEA':'Seahawks','MIA':'Dolphins','MIN':'Vikings',
                                   'NE':'Patriots','NYG':'Giants','TEN':'Titans',
                                   'DEN':'Broncos', 'NYJ':'Jets','PHI':'Eagles',
                                   'LAC':'Chargers','TB':'Buccaneers','BUF':'Bills',
                                   'CLE':'Browns','ATL':'Falcons','JAX':'Jaguars',
                                   'CHI':'Bears','GB':'Packers'})
    
    nfl_sched['Sack'] = None
    nfl_sched['FR'] = None
    nfl_sched['INT'] = None
    nfl_sched['DefTD'] = None
    nfl_sched['PA'] = None
    nfl_sched['PaYD'] = None
    nfl_sched['RuYd'] = None
    nfl_sched['Safety'] = None
    nfl_sched['KickTD'] = None
    nfl_sched['FPts'] = None
    nfl_sched['away flag'] = None

    def_df['Opp'] = def_df['Opp'].str.replace('@','')
    def_df['opp_wk_year'] = def_df['Opp'] + " " + def_df['week_num'].astype(str) + " " + def_df['year'].astype(str)
    def_df = def_df.append(nfl_sched)

#    filt_wk = str(curr_wk) + " " + str(curr_yr)
    
#    def_df_wk = def_df_wk[def_df_wk['team_wk_year'].str.contains(filt_wk)]
    
    return def_df